In [1]:
import pandas as pd
import numpy as np
from datetime import datetime as dt

In [74]:
pd.options.display.max_rows = 200
pd.options.display.max_columns = 40
#pd.set_option('display.max_rows', 200)

In [2]:
dataName = 'WCup2022_TeamData.csv'
DFTeamData = pd.read_csv(dataName)
DFTeamData

,Team,Group,Area,Ranking
0,Qatar,A,Host,50
1,Ecuador,A,SouthAmerica,44
2,Senegal,A,Africa,18
3,Netherlands,A,Europe,8
4,England,B,Europe,5
5,Iran,B,Asia,20
6,USA,B,Caribbean,16
7,Wales,B,Europe,19
8,Argentina,C,SouthAmerica,3
9,SaudiArabia,C,Asia,51


In [3]:
def Ranking(RoundRobin):
    Team_list = RoundRobin.index.tolist()
    
    for team_index in Team_list:
        scored_sum = 0
        conceded_sum = 0
        winningPoint = 0
        for team_columns in Team_list:
            score = RoundRobin.at[team_index, team_columns]
            if type(score) is float:
                continue
            score_list = score.split(',')
            if len(score_list) != 2:
                continue
            
            scored, conceded = int(score_list[0]), int(score_list[1])
            scored_sum += scored
            conceded_sum += conceded
            if scored > conceded: winningPoint += 3
            elif scored == conceded: winningPoint += 1
                
        RoundRobin.at[team_index, 'WinningPoints'] = winningPoint
        RoundRobin.at[team_index, 'Score'] = scored_sum
        RoundRobin.at[team_index, 'Conceded'] = conceded_sum
        RoundRobin.at[team_index, 'GoalDiff'] = scored_sum - conceded_sum
        
    WinningPoints_rank = RoundRobin['WinningPoints'].rank(ascending=False, method='min')
    WinningPoints_list = WinningPoints_rank.tolist()
    RoundRobin['Ranking'] = WinningPoints_list
    Duplication_list1 = []
    for i in range(1,5):
        if WinningPoints_list.count(i) >= 2:
            Duplication_list1.append(i)
    for duplication1 in Duplication_list1:
        RoundRobin_duplication1 = RoundRobin.loc[RoundRobin['Ranking'] == duplication1]
        GoalDiff_rank = RoundRobin_duplication1['GoalDiff'].rank(ascending=False, method='min')
        GoalDiff_list = GoalDiff_rank.tolist()
        GoalDiff_index = GoalDiff_rank.index.tolist()
        for j in range(1,len(GoalDiff_list)+1):
            if GoalDiff_list.count(j) == 1:
                RoundRobin.at[GoalDiff_index[GoalDiff_list.index(j)], 'Ranking'] = j + duplication1-1
            elif GoalDiff_list.count(j) >= 2:
                idx = -1
                Duplication_list2 = []
                for cnt in range(GoalDiff_list.count(j)):
                    idx = GoalDiff_list.index(j, idx+1)
                    Duplication_list2.append(GoalDiff_index[idx])
                RoundRobin.loc[Duplication_list2, 'Ranking'] = j + duplication1-1
                RoundRobin_duplication2 = RoundRobin.loc[Duplication_list2]
                Score_rank = RoundRobin_duplication2['Score'].rank(ascending=False, method='min')
                Score_list = Score_rank.tolist()
                Score_index = Score_rank.index.tolist()
                for k in range(1,len(Score_list)+1):
                    if Score_list.count(k) == 1:
                        RoundRobin.at[Score_index[Score_list.index(k)], 'Ranking'] = k + j-1 + duplication1-1
    
    
    return RoundRobin

In [4]:
Score_list        = ['7,0'
                     , '7,1', '6,0'
                     , '7,2', '6,1', '5,0'
                     , '7,3', '6,2', '5,1', '4,0'
                     , '7,4', '6,3', '5,2', '4,1', '3,0'
                     , '7,5', '6,4', '5,3', '4,2', '3,1', '2,0'
                     , '7,6', '6,5', '5,4', '4,3', '3,2', '2,1', '1,0'
                     , '7,7', '6,6', '5,5', '4,4', '3,3', '2,2', '1,1', '0,0'
                     , '6,7', '5,6', '4,5', '3,4', '2,3', '1,2', '0,1'
                     , '5,7', '4,6', '3,5', '2,4', '1,3', '0,2'
                     , '4,7', '3,6', '2,5', '1,4', '0,3'
                     , '3,7', '2.6', '1,5', '0,4'
                     , '2,7', '1,6', '0,5'
                     , '1,7', '0,6'
                     , '0,7']
Score_inverselist = ['0,7'
                     , '1,7', '0,6'
                     , '2,7', '1,6', '0,5'
                     , '3,7', '2.6', '1,5', '0,4'
                     , '4,7', '3,6', '2,5', '1,4', '0,3'
                     , '5,7', '4,6', '3,5', '2,4', '1,3', '0,2'
                     , '6,7', '5,6', '4,5', '3,4', '2,3', '1,2', '0,1'
                     , '7,7', '6,6', '5,5', '4,4', '3,3', '2,2', '1,1', '0,0'
                     , '7,6', '6,5', '5,4', '4,3', '3,2', '2,1', '1,0'
                     , '7,5', '6,4', '5,3', '4,2', '3,1', '2,0'
                     , '7,4', '6,3', '5,2', '4,1', '3,0'
                     , '7,3', '6,2', '5,1', '4,0'
                     , '7,2', '6,1', '5,0'
                     , '7,1', '6,0'
                     , '7,0']

In [5]:
def RankingProbability(RoundRobin):
    Team_list = RoundRobin.index.tolist()
    #Score_list = ['5,0', '4,0', '3,0', '2,0', '1,0', '0,0', '0,1', '0,2', '0,3', '0,4', '0,5']
    #Score_list = ['1,0', '0,0', '0,1']
    # 11パターンで240秒(7分), 1秒で約670回
    # 予測：5試合分　13パターンで555秒x13=2時間
    # 予測：3試合分　55パターンで250秒x55=3.8時間, 64パターンで360秒x64=384分(6.4時間)

    X = np.array(range(len(Score_list)))
    
    Ranking_num = pd.DataFrame(0, columns=range(1,5), index=Team_list)
    Ranking_probability = pd.DataFrame(columns=range(1,5), index=Team_list)
    print("start : " + dt.now().strftime("%Y.%m.%d %H:%M:%S"))
    continueSwitch1 = 0
    for i1 in X:
        if continueSwitch1 == 1: continue
        #print("0,1(" + str(i1) + ") : " + dt.now().strftime("%Y.%m.%d %H:%M:%S"))
        RoundRobin1 = RoundRobin.copy()
        if RoundRobin1.iat[0,1] is float(np.nan):
            RoundRobin1.iat[0,1] = Score_list[i1]
            RoundRobin1.iat[1,0] = Score_inverselist[i1]
        else: continueSwitch1 = 1
        continueSwitch2 = 0
        for i2 in X:
            if continueSwitch2 == 1: continue
            #print("0,2(" + str(i2) + ") : " + dt.now().strftime("%Y.%m.%d %H:%M:%S"))
            RoundRobin2 = RoundRobin1.copy()
            if RoundRobin2.iat[0,2] is float(np.nan):
                RoundRobin2.iat[0,2] = Score_list[i2]
                RoundRobin2.iat[2,0] = Score_inverselist[i2]
            else: continueSwitch2 = 1
            continueSwitch3 = 0
            for i3 in X:
                if continueSwitch3 == 1: continue
                #print("0,3(" + str(i3) + ") : " + dt.now().strftime("%Y.%m.%d %H:%M:%S"))
                RoundRobin3 = RoundRobin2.copy()
                if RoundRobin3.iat[0,3] is float(np.nan):
                    RoundRobin3.iat[0,3] = Score_list[i3]
                    RoundRobin3.iat[3,0] = Score_inverselist[i3]
                else: continueSwitch3 = 1
                continueSwitch4 = 0
                for i4 in X:
                    if continueSwitch4 == 1: continue
                    #print("1,2 : " + dt.now().strftime("%Y.%m.%d %H:%M:%S"))
                    RoundRobin4 = RoundRobin3.copy()
                    if RoundRobin4.iat[1,2] is float(np.nan):
                        RoundRobin4.iat[1,2] = Score_list[i4]
                        RoundRobin4.iat[2,1] = Score_inverselist[i4]
                    else: continueSwitch4 = 1
                    continueSwitch5 = 0
                    for i5 in X:
                        if continueSwitch5 == 1: continue
                        #print("1,3 : " + dt.now().strftime("%Y.%m.%d %H:%M:%S"))
                        RoundRobin5 = RoundRobin4.copy()
                        if RoundRobin5.iat[1,3] is float(np.nan):
                            RoundRobin5.iat[1,3] = Score_list[i5]
                            RoundRobin5.iat[3,1] = Score_inverselist[i5]
                        else: continueSwitch5 = 1
                        continueSwitch6 = 0
                        for i6 in X:
                            if continueSwitch6 == 1: continue
                            #print("2,3 : " + dt.now().strftime("%Y.%m.%d %H:%M:%S"))
                            RoundRobin6 = RoundRobin5.copy()
                            if RoundRobin6.iat[2,3] is float(np.nan):
                                RoundRobin6.iat[2,3] = Score_list[i6]
                                RoundRobin6.iat[3,2] = Score_inverselist[i6]
                            else: continueSwitch6 = 1
                                
                            RoundRobin_rank = Ranking(RoundRobin6.copy())
                            
                            for j in range(1,5):
                                for k in range(0,4):
                                    if RoundRobin_rank.at[Team_list[k], 'Ranking'] == j:
                                        Ranking_num.at[Team_list[k], j] += 1
                            
    
    for k in range(0,4):
        patternNum = Ranking_num.loc[Team_list[k], :].sum()
        for j in range(1,5):
            Ranking_probability.at[Team_list[k], j] = str((Ranking_num.at[Team_list[k], j]/patternNum*100).round(1)) + '%'
        Ranking_probability.at[Team_list[k], 'Breakthrough'] = str(((Ranking_num.at[Team_list[k], 1]+Ranking_num.at[Team_list[k], 2])/patternNum*100).round(1)) + '%'
                            
    return Ranking_num, Ranking_probability

In [6]:
Group_E_Team = DFTeamData.loc[DFTeamData['Group'] == 'E']
Group_E_Team_list = Group_E_Team['Team'].tolist()
Group_E_RoundRobin = pd.DataFrame(columns=Group_E_Team_list, index=Group_E_Team_list)
for team in Group_E_Team_list:
    Group_E_RoundRobin.at[team, team] = "-"

In [7]:
Ranking_JapanPropability = pd.DataFrame(columns=range(1,5))

In [8]:
no = 0
Group_E_RoundRobin.iat[0, 1], Group_E_RoundRobin.iat[1, 0] = '7,0' , '0,7'
Group_E_RoundRobin.iat[2, 3], Group_E_RoundRobin.iat[3, 2] = '1,2' , '2,1'

#Group_E_RoundRobin.iat[0, 2], Group_E_RoundRobin.iat[2, 0] = '0,2' , '2,0'
Group_E_RoundRobin.iat[1, 3], Group_E_RoundRobin.iat[3, 1] = Score_inverselist[no] , Score_list[no]

#Group_E_RoundRobin.iat[0, 3], Group_E_RoundRobin.iat[3, 0] = '0,2' , '2,0'
#Group_E_RoundRobin.iat[1, 2], Group_E_RoundRobin.iat[2, 1] = '0,2' , '2,0'
Ranking(Group_E_RoundRobin)
Ranking_num_E, Ranking_probability_E = RankingProbability(Group_E_RoundRobin.copy())
for i in range(1,5):
    Ranking_JapanPropability.at[Score_list[no], i] = Ranking_probability_E.at['Japan', i]
Ranking_JapanPropability.at[Score_list[no], 'Breakthrough'] = Ranking_probability_E.at['Japan', 'Breakthrough']
print(Score_list[no])
Ranking_probability_E

start : 2022.11.24 18:43:28
7,0


,1,2,3,4,Breakthrough
Spain,42.8%,39.7%,17.5%,0.0%,82.5%
CostaRica,0.0%,0.0%,24.8%,75.2%,0.0%
Germany,1.8%,18.8%,54.7%,24.7%,20.6%
Japan,56.0%,41.2%,2.8%,0.0%,97.2%


In [10]:
no = 1
Group_E_RoundRobin.iat[0, 1], Group_E_RoundRobin.iat[1, 0] = '7,0' , '0,7'
Group_E_RoundRobin.iat[2, 3], Group_E_RoundRobin.iat[3, 2] = '1,2' , '2,1'

#Group_E_RoundRobin.iat[0, 2], Group_E_RoundRobin.iat[2, 0] = '0,2' , '2,0'
Group_E_RoundRobin.iat[1, 3], Group_E_RoundRobin.iat[3, 1] = Score_inverselist[no] , Score_list[no]

#Group_E_RoundRobin.iat[0, 3], Group_E_RoundRobin.iat[3, 0] = '0,2' , '2,0'
#Group_E_RoundRobin.iat[1, 2], Group_E_RoundRobin.iat[2, 1] = '0,2' , '2,0'
Ranking(Group_E_RoundRobin)
Ranking_num_E, Ranking_probability_E = RankingProbability(Group_E_RoundRobin.copy())
for i in range(1,5):
    Ranking_JapanPropability.at[Score_list[no], i] = Ranking_probability_E.at['Japan', i]
Ranking_JapanPropability.at[Score_list[no], 'Breakthrough'] = Ranking_probability_E.at['Japan', 'Breakthrough']
print(Score_list[no])
Ranking_probability_E

start : 2022.11.24 19:23:15
7,1


,1,2,3,4,Breakthrough
Spain,44.5%,38.2%,17.3%,0.0%,82.7%
CostaRica,0.0%,0.0%,24.9%,75.1%,0.0%
Germany,2.1%,19.2%,53.8%,24.9%,21.3%
Japan,53.8%,42.5%,3.7%,0.0%,96.3%


In [11]:
no = 2
Group_E_RoundRobin.iat[0, 1], Group_E_RoundRobin.iat[1, 0] = '7,0' , '0,7'
Group_E_RoundRobin.iat[2, 3], Group_E_RoundRobin.iat[3, 2] = '1,2' , '2,1'

#Group_E_RoundRobin.iat[0, 2], Group_E_RoundRobin.iat[2, 0] = '0,2' , '2,0'
Group_E_RoundRobin.iat[1, 3], Group_E_RoundRobin.iat[3, 1] = Score_inverselist[no] , Score_list[no]

#Group_E_RoundRobin.iat[0, 3], Group_E_RoundRobin.iat[3, 0] = '0,2' , '2,0'
#Group_E_RoundRobin.iat[1, 2], Group_E_RoundRobin.iat[2, 1] = '0,2' , '2,0'
Ranking(Group_E_RoundRobin)
Ranking_num_E, Ranking_probability_E = RankingProbability(Group_E_RoundRobin.copy())
for i in range(1,5):
    Ranking_JapanPropability.at[Score_list[no], i] = Ranking_probability_E.at['Japan', i]
Ranking_JapanPropability.at[Score_list[no], 'Breakthrough'] = Ranking_probability_E.at['Japan', 'Breakthrough']
print(Score_list[no])
Ranking_probability_E

start : 2022.11.24 19:29:13
6,0


,1,2,3,4,Breakthrough
Spain,44.7%,38.1%,17.3%,0.0%,82.7%
CostaRica,0.0%,0.0%,24.9%,75.1%,0.0%
Germany,2.1%,19.3%,53.7%,24.9%,21.4%
Japan,53.5%,42.6%,3.9%,0.0%,96.1%


In [12]:
no = 3
Group_E_RoundRobin.iat[0, 1], Group_E_RoundRobin.iat[1, 0] = '7,0' , '0,7'
Group_E_RoundRobin.iat[2, 3], Group_E_RoundRobin.iat[3, 2] = '1,2' , '2,1'

#Group_E_RoundRobin.iat[0, 2], Group_E_RoundRobin.iat[2, 0] = '0,2' , '2,0'
Group_E_RoundRobin.iat[1, 3], Group_E_RoundRobin.iat[3, 1] = Score_inverselist[no] , Score_list[no]

#Group_E_RoundRobin.iat[0, 3], Group_E_RoundRobin.iat[3, 0] = '0,2' , '2,0'
#Group_E_RoundRobin.iat[1, 2], Group_E_RoundRobin.iat[2, 1] = '0,2' , '2,0'
Ranking(Group_E_RoundRobin)
Ranking_num_E, Ranking_probability_E = RankingProbability(Group_E_RoundRobin.copy())
for i in range(1,5):
    Ranking_JapanPropability.at[Score_list[no], i] = Ranking_probability_E.at['Japan', i]
Ranking_JapanPropability.at[Score_list[no], 'Breakthrough'] = Ranking_probability_E.at['Japan', 'Breakthrough']
print(Score_list[no])
Ranking_probability_E

start : 2022.11.24 19:35:10
7,2


,1,2,3,4,Breakthrough
Spain,45.6%,37.5%,16.9%,0.0%,83.1%
CostaRica,0.0%,0.0%,25.4%,74.6%,0.0%
Germany,2.3%,19.8%,52.6%,25.3%,22.1%
Japan,52.4%,42.7%,4.9%,0.0%,95.1%


In [13]:
no = 4
Group_E_RoundRobin.iat[0, 1], Group_E_RoundRobin.iat[1, 0] = '7,0' , '0,7'
Group_E_RoundRobin.iat[2, 3], Group_E_RoundRobin.iat[3, 2] = '1,2' , '2,1'

#Group_E_RoundRobin.iat[0, 2], Group_E_RoundRobin.iat[2, 0] = '0,2' , '2,0'
Group_E_RoundRobin.iat[1, 3], Group_E_RoundRobin.iat[3, 1] = Score_inverselist[no] , Score_list[no]

#Group_E_RoundRobin.iat[0, 3], Group_E_RoundRobin.iat[3, 0] = '0,2' , '2,0'
#Group_E_RoundRobin.iat[1, 2], Group_E_RoundRobin.iat[2, 1] = '0,2' , '2,0'
Ranking(Group_E_RoundRobin)
Ranking_num_E, Ranking_probability_E = RankingProbability(Group_E_RoundRobin.copy())
for i in range(1,5):
    Ranking_JapanPropability.at[Score_list[no], i] = Ranking_probability_E.at['Japan', i]
Ranking_JapanPropability.at[Score_list[no], 'Breakthrough'] = Ranking_probability_E.at['Japan', 'Breakthrough']
print(Score_list[no])
Ranking_probability_E

start : 2022.11.24 19:41:04
6,1


,1,2,3,4,Breakthrough
Spain,45.8%,37.4%,16.8%,0.0%,83.2%
CostaRica,0.0%,0.0%,25.4%,74.6%,0.0%
Germany,2.3%,19.9%,52.6%,25.3%,22.2%
Japan,52.2%,42.7%,5.1%,0.0%,94.9%


In [14]:
no = 5
Group_E_RoundRobin.iat[0, 1], Group_E_RoundRobin.iat[1, 0] = '7,0' , '0,7'
Group_E_RoundRobin.iat[2, 3], Group_E_RoundRobin.iat[3, 2] = '1,2' , '2,1'

#Group_E_RoundRobin.iat[0, 2], Group_E_RoundRobin.iat[2, 0] = '0,2' , '2,0'
Group_E_RoundRobin.iat[1, 3], Group_E_RoundRobin.iat[3, 1] = Score_inverselist[no] , Score_list[no]

#Group_E_RoundRobin.iat[0, 3], Group_E_RoundRobin.iat[3, 0] = '0,2' , '2,0'
#Group_E_RoundRobin.iat[1, 2], Group_E_RoundRobin.iat[2, 1] = '0,2' , '2,0'
Ranking(Group_E_RoundRobin)
Ranking_num_E, Ranking_probability_E = RankingProbability(Group_E_RoundRobin.copy())
for i in range(1,5):
    Ranking_JapanPropability.at[Score_list[no], i] = Ranking_probability_E.at['Japan', i]
Ranking_JapanPropability.at[Score_list[no], 'Breakthrough'] = Ranking_probability_E.at['Japan', 'Breakthrough']
print(Score_list[no])
Ranking_probability_E

start : 2022.11.24 19:46:58
5,0


,1,2,3,4,Breakthrough
Spain,45.8%,37.4%,16.8%,0.0%,83.2%
CostaRica,0.0%,0.0%,25.3%,74.7%,0.0%
Germany,2.3%,20.0%,52.5%,25.2%,22.3%
Japan,52.0%,42.8%,5.2%,0.0%,94.8%


In [15]:
no = 6
Group_E_RoundRobin.iat[0, 1], Group_E_RoundRobin.iat[1, 0] = '7,0' , '0,7'
Group_E_RoundRobin.iat[2, 3], Group_E_RoundRobin.iat[3, 2] = '1,2' , '2,1'

#Group_E_RoundRobin.iat[0, 2], Group_E_RoundRobin.iat[2, 0] = '0,2' , '2,0'
Group_E_RoundRobin.iat[1, 3], Group_E_RoundRobin.iat[3, 1] = Score_inverselist[no] , Score_list[no]

#Group_E_RoundRobin.iat[0, 3], Group_E_RoundRobin.iat[3, 0] = '0,2' , '2,0'
#Group_E_RoundRobin.iat[1, 2], Group_E_RoundRobin.iat[2, 1] = '0,2' , '2,0'
Ranking(Group_E_RoundRobin)
Ranking_num_E, Ranking_probability_E = RankingProbability(Group_E_RoundRobin.copy())
for i in range(1,5):
    Ranking_JapanPropability.at[Score_list[no], i] = Ranking_probability_E.at['Japan', i]
Ranking_JapanPropability.at[Score_list[no], 'Breakthrough'] = Ranking_probability_E.at['Japan', 'Breakthrough']
print(Score_list[no])
Ranking_probability_E

start : 2022.11.24 19:52:54
7,3


,1,2,3,4,Breakthrough
Spain,46.2%,37.2%,16.6%,0.0%,83.4%
CostaRica,0.0%,0.0%,25.8%,74.2%,0.0%
Germany,2.3%,20.5%,51.4%,25.8%,22.8%
Japan,51.8%,42.3%,5.9%,0.0%,94.1%


In [16]:
no = 7
Group_E_RoundRobin.iat[0, 1], Group_E_RoundRobin.iat[1, 0] = '7,0' , '0,7'
Group_E_RoundRobin.iat[2, 3], Group_E_RoundRobin.iat[3, 2] = '1,2' , '2,1'

#Group_E_RoundRobin.iat[0, 2], Group_E_RoundRobin.iat[2, 0] = '0,2' , '2,0'
Group_E_RoundRobin.iat[1, 3], Group_E_RoundRobin.iat[3, 1] = Score_inverselist[no] , Score_list[no]

#Group_E_RoundRobin.iat[0, 3], Group_E_RoundRobin.iat[3, 0] = '0,2' , '2,0'
#Group_E_RoundRobin.iat[1, 2], Group_E_RoundRobin.iat[2, 1] = '0,2' , '2,0'
Ranking(Group_E_RoundRobin)
Ranking_num_E, Ranking_probability_E = RankingProbability(Group_E_RoundRobin.copy())
for i in range(1,5):
    Ranking_JapanPropability.at[Score_list[no], i] = Ranking_probability_E.at['Japan', i]
Ranking_JapanPropability.at[Score_list[no], 'Breakthrough'] = Ranking_probability_E.at['Japan', 'Breakthrough']
print(Score_list[no])
Ranking_probability_E

start : 2022.11.24 19:58:47
6,2


,1,2,3,4,Breakthrough
Spain,46.3%,37.1%,16.4%,0.2%,83.4%
CostaRica,0.0%,0.1%,28.8%,71.0%,0.1%
Germany,2.3%,20.6%,48.5%,28.6%,22.9%
Japan,51.6%,42.3%,6.1%,0.0%,93.9%


In [17]:
no = 8
Group_E_RoundRobin.iat[0, 1], Group_E_RoundRobin.iat[1, 0] = '7,0' , '0,7'
Group_E_RoundRobin.iat[2, 3], Group_E_RoundRobin.iat[3, 2] = '1,2' , '2,1'

#Group_E_RoundRobin.iat[0, 2], Group_E_RoundRobin.iat[2, 0] = '0,2' , '2,0'
Group_E_RoundRobin.iat[1, 3], Group_E_RoundRobin.iat[3, 1] = Score_inverselist[no] , Score_list[no]

#Group_E_RoundRobin.iat[0, 3], Group_E_RoundRobin.iat[3, 0] = '0,2' , '2,0'
#Group_E_RoundRobin.iat[1, 2], Group_E_RoundRobin.iat[2, 1] = '0,2' , '2,0'
Ranking(Group_E_RoundRobin)
Ranking_num_E, Ranking_probability_E = RankingProbability(Group_E_RoundRobin.copy())
for i in range(1,5):
    Ranking_JapanPropability.at[Score_list[no], i] = Ranking_probability_E.at['Japan', i]
Ranking_JapanPropability.at[Score_list[no], 'Breakthrough'] = Ranking_probability_E.at['Japan', 'Breakthrough']
print(Score_list[no])
Ranking_probability_E

start : 2022.11.24 20:04:43
5,1


,1,2,3,4,Breakthrough
Spain,46.3%,37.2%,16.5%,0.0%,83.5%
CostaRica,0.0%,0.0%,25.8%,74.2%,0.0%
Germany,2.3%,20.7%,51.3%,25.7%,23.0%
Japan,51.5%,42.2%,6.3%,0.0%,93.7%


In [18]:
no = 9
Group_E_RoundRobin.iat[0, 1], Group_E_RoundRobin.iat[1, 0] = '7,0' , '0,7'
Group_E_RoundRobin.iat[2, 3], Group_E_RoundRobin.iat[3, 2] = '1,2' , '2,1'

#Group_E_RoundRobin.iat[0, 2], Group_E_RoundRobin.iat[2, 0] = '0,2' , '2,0'
Group_E_RoundRobin.iat[1, 3], Group_E_RoundRobin.iat[3, 1] = Score_inverselist[no] , Score_list[no]

#Group_E_RoundRobin.iat[0, 3], Group_E_RoundRobin.iat[3, 0] = '0,2' , '2,0'
#Group_E_RoundRobin.iat[1, 2], Group_E_RoundRobin.iat[2, 1] = '0,2' , '2,0'
Ranking(Group_E_RoundRobin)
Ranking_num_E, Ranking_probability_E = RankingProbability(Group_E_RoundRobin.copy())
for i in range(1,5):
    Ranking_JapanPropability.at[Score_list[no], i] = Ranking_probability_E.at['Japan', i]
Ranking_JapanPropability.at[Score_list[no], 'Breakthrough'] = Ranking_probability_E.at['Japan', 'Breakthrough']
print(Score_list[no])
Ranking_probability_E

start : 2022.11.24 20:10:36
4,0


,1,2,3,4,Breakthrough
Spain,46.3%,37.2%,16.5%,0.0%,83.5%
CostaRica,0.0%,0.0%,25.7%,74.3%,0.0%
Germany,2.3%,20.8%,51.3%,25.6%,23.1%
Japan,51.5%,42.1%,6.4%,0.0%,93.6%


In [19]:
no = 10
Group_E_RoundRobin.iat[0, 1], Group_E_RoundRobin.iat[1, 0] = '7,0' , '0,7'
Group_E_RoundRobin.iat[2, 3], Group_E_RoundRobin.iat[3, 2] = '1,2' , '2,1'

#Group_E_RoundRobin.iat[0, 2], Group_E_RoundRobin.iat[2, 0] = '0,2' , '2,0'
Group_E_RoundRobin.iat[1, 3], Group_E_RoundRobin.iat[3, 1] = Score_inverselist[no] , Score_list[no]

#Group_E_RoundRobin.iat[0, 3], Group_E_RoundRobin.iat[3, 0] = '0,2' , '2,0'
#Group_E_RoundRobin.iat[1, 2], Group_E_RoundRobin.iat[2, 1] = '0,2' , '2,0'
Ranking(Group_E_RoundRobin)
Ranking_num_E, Ranking_probability_E = RankingProbability(Group_E_RoundRobin.copy())
for i in range(1,5):
    Ranking_JapanPropability.at[Score_list[no], i] = Ranking_probability_E.at['Japan', i]
Ranking_JapanPropability.at[Score_list[no], 'Breakthrough'] = Ranking_probability_E.at['Japan', 'Breakthrough']
print(Score_list[no])
Ranking_probability_E

start : 2022.11.24 20:16:30
7,4


,1,2,3,4,Breakthrough
Spain,46.6%,37.1%,16.2%,0.1%,83.7%
CostaRica,0.0%,0.0%,26.7%,73.3%,0.0%
Germany,2.3%,21.2%,49.9%,26.6%,23.5%
Japan,51.3%,41.7%,6.9%,0.0%,93.1%


In [20]:
no = 11
Group_E_RoundRobin.iat[0, 1], Group_E_RoundRobin.iat[1, 0] = '7,0' , '0,7'
Group_E_RoundRobin.iat[2, 3], Group_E_RoundRobin.iat[3, 2] = '1,2' , '2,1'

#Group_E_RoundRobin.iat[0, 2], Group_E_RoundRobin.iat[2, 0] = '0,2' , '2,0'
Group_E_RoundRobin.iat[1, 3], Group_E_RoundRobin.iat[3, 1] = Score_inverselist[no] , Score_list[no]

#Group_E_RoundRobin.iat[0, 3], Group_E_RoundRobin.iat[3, 0] = '0,2' , '2,0'
#Group_E_RoundRobin.iat[1, 2], Group_E_RoundRobin.iat[2, 1] = '0,2' , '2,0'
Ranking(Group_E_RoundRobin)
Ranking_num_E, Ranking_probability_E = RankingProbability(Group_E_RoundRobin.copy())
for i in range(1,5):
    Ranking_JapanPropability.at[Score_list[no], i] = Ranking_probability_E.at['Japan', i]
Ranking_JapanPropability.at[Score_list[no], 'Breakthrough'] = Ranking_probability_E.at['Japan', 'Breakthrough']
print(Score_list[no])
Ranking_probability_E

start : 2022.11.24 20:22:25
6,3


,1,2,3,4,Breakthrough
Spain,46.7%,37.1%,16.2%,0.1%,83.8%
CostaRica,0.0%,0.0%,26.7%,73.3%,0.0%
Germany,2.3%,21.3%,49.9%,26.5%,23.6%
Japan,51.2%,41.7%,7.1%,0.0%,92.9%


In [21]:
no = 12
Group_E_RoundRobin.iat[0, 1], Group_E_RoundRobin.iat[1, 0] = '7,0' , '0,7'
Group_E_RoundRobin.iat[2, 3], Group_E_RoundRobin.iat[3, 2] = '1,2' , '2,1'

#Group_E_RoundRobin.iat[0, 2], Group_E_RoundRobin.iat[2, 0] = '0,2' , '2,0'
Group_E_RoundRobin.iat[1, 3], Group_E_RoundRobin.iat[3, 1] = Score_inverselist[no] , Score_list[no]

#Group_E_RoundRobin.iat[0, 3], Group_E_RoundRobin.iat[3, 0] = '0,2' , '2,0'
#Group_E_RoundRobin.iat[1, 2], Group_E_RoundRobin.iat[2, 1] = '0,2' , '2,0'
Ranking(Group_E_RoundRobin)
Ranking_num_E, Ranking_probability_E = RankingProbability(Group_E_RoundRobin.copy())
for i in range(1,5):
    Ranking_JapanPropability.at[Score_list[no], i] = Ranking_probability_E.at['Japan', i]
Ranking_JapanPropability.at[Score_list[no], 'Breakthrough'] = Ranking_probability_E.at['Japan', 'Breakthrough']
print(Score_list[no])
Ranking_probability_E

start : 2022.11.24 20:28:21
5,2


,1,2,3,4,Breakthrough
Spain,46.7%,37.1%,16.2%,0.0%,83.8%
CostaRica,0.0%,0.0%,26.6%,73.4%,0.0%
Germany,2.3%,21.3%,49.9%,26.4%,23.7%
Japan,51.1%,41.7%,7.3%,0.0%,92.7%


In [22]:
no = 13
Group_E_RoundRobin.iat[0, 1], Group_E_RoundRobin.iat[1, 0] = '7,0' , '0,7'
Group_E_RoundRobin.iat[2, 3], Group_E_RoundRobin.iat[3, 2] = '1,2' , '2,1'

#Group_E_RoundRobin.iat[0, 2], Group_E_RoundRobin.iat[2, 0] = '0,2' , '2,0'
Group_E_RoundRobin.iat[1, 3], Group_E_RoundRobin.iat[3, 1] = Score_inverselist[no] , Score_list[no]

#Group_E_RoundRobin.iat[0, 3], Group_E_RoundRobin.iat[3, 0] = '0,2' , '2,0'
#Group_E_RoundRobin.iat[1, 2], Group_E_RoundRobin.iat[2, 1] = '0,2' , '2,0'
Ranking(Group_E_RoundRobin)
Ranking_num_E, Ranking_probability_E = RankingProbability(Group_E_RoundRobin.copy())
for i in range(1,5):
    Ranking_JapanPropability.at[Score_list[no], i] = Ranking_probability_E.at['Japan', i]
Ranking_JapanPropability.at[Score_list[no], 'Breakthrough'] = Ranking_probability_E.at['Japan', 'Breakthrough']
print(Score_list[no])
Ranking_probability_E

start : 2022.11.24 20:34:15
4,1


,1,2,3,4,Breakthrough
Spain,46.7%,37.1%,16.2%,0.0%,83.8%
CostaRica,0.0%,0.0%,26.4%,73.5%,0.0%
Germany,2.3%,21.4%,50.0%,26.3%,23.7%
Japan,51.1%,41.6%,7.3%,0.0%,92.7%


In [23]:
no = 14
Group_E_RoundRobin.iat[0, 1], Group_E_RoundRobin.iat[1, 0] = '7,0' , '0,7'
Group_E_RoundRobin.iat[2, 3], Group_E_RoundRobin.iat[3, 2] = '1,2' , '2,1'

#Group_E_RoundRobin.iat[0, 2], Group_E_RoundRobin.iat[2, 0] = '0,2' , '2,0'
Group_E_RoundRobin.iat[1, 3], Group_E_RoundRobin.iat[3, 1] = Score_inverselist[no] , Score_list[no]

#Group_E_RoundRobin.iat[0, 3], Group_E_RoundRobin.iat[3, 0] = '0,2' , '2,0'
#Group_E_RoundRobin.iat[1, 2], Group_E_RoundRobin.iat[2, 1] = '0,2' , '2,0'
Ranking(Group_E_RoundRobin)
Ranking_num_E, Ranking_probability_E = RankingProbability(Group_E_RoundRobin.copy())
for i in range(1,5):
    Ranking_JapanPropability.at[Score_list[no], i] = Ranking_probability_E.at['Japan', i]
Ranking_JapanPropability.at[Score_list[no], 'Breakthrough'] = Ranking_probability_E.at['Japan', 'Breakthrough']
print(Score_list[no])
Ranking_probability_E

start : 2022.11.24 21:12:38
3,0


,1,2,3,4,Breakthrough
Spain,46.7%,37.1%,16.2%,0.0%,83.8%
CostaRica,0.0%,0.0%,26.3%,73.7%,0.0%
Germany,2.3%,21.5%,50.1%,26.1%,23.8%
Japan,51.1%,41.5%,7.4%,0.0%,92.6%


In [24]:
no = 15
Group_E_RoundRobin.iat[0, 1], Group_E_RoundRobin.iat[1, 0] = '7,0' , '0,7'
Group_E_RoundRobin.iat[2, 3], Group_E_RoundRobin.iat[3, 2] = '1,2' , '2,1'

#Group_E_RoundRobin.iat[0, 2], Group_E_RoundRobin.iat[2, 0] = '0,2' , '2,0'
Group_E_RoundRobin.iat[1, 3], Group_E_RoundRobin.iat[3, 1] = Score_inverselist[no] , Score_list[no]

#Group_E_RoundRobin.iat[0, 3], Group_E_RoundRobin.iat[3, 0] = '0,2' , '2,0'
#Group_E_RoundRobin.iat[1, 2], Group_E_RoundRobin.iat[2, 1] = '0,2' , '2,0'
Ranking(Group_E_RoundRobin)
Ranking_num_E, Ranking_probability_E = RankingProbability(Group_E_RoundRobin.copy())
for i in range(1,5):
    Ranking_JapanPropability.at[Score_list[no], i] = Ranking_probability_E.at['Japan', i]
Ranking_JapanPropability.at[Score_list[no], 'Breakthrough'] = Ranking_probability_E.at['Japan', 'Breakthrough']
print(Score_list[no])
Ranking_probability_E

start : 2022.11.24 21:18:38
7,5


,1,2,3,4,Breakthrough
Spain,46.8%,37.1%,16.1%,0.1%,83.8%
CostaRica,0.0%,0.0%,27.4%,72.6%,0.0%
Germany,2.3%,21.7%,48.7%,27.3%,24.0%
Japan,51.1%,41.3%,7.7%,0.0%,92.3%


In [25]:
no = 16
Group_E_RoundRobin.iat[0, 1], Group_E_RoundRobin.iat[1, 0] = '7,0' , '0,7'
Group_E_RoundRobin.iat[2, 3], Group_E_RoundRobin.iat[3, 2] = '1,2' , '2,1'

#Group_E_RoundRobin.iat[0, 2], Group_E_RoundRobin.iat[2, 0] = '0,2' , '2,0'
Group_E_RoundRobin.iat[1, 3], Group_E_RoundRobin.iat[3, 1] = Score_inverselist[no] , Score_list[no]

#Group_E_RoundRobin.iat[0, 3], Group_E_RoundRobin.iat[3, 0] = '0,2' , '2,0'
#Group_E_RoundRobin.iat[1, 2], Group_E_RoundRobin.iat[2, 1] = '0,2' , '2,0'
Ranking(Group_E_RoundRobin)
Ranking_num_E, Ranking_probability_E = RankingProbability(Group_E_RoundRobin.copy())
for i in range(1,5):
    Ranking_JapanPropability.at[Score_list[no], i] = Ranking_probability_E.at['Japan', i]
Ranking_JapanPropability.at[Score_list[no], 'Breakthrough'] = Ranking_probability_E.at['Japan', 'Breakthrough']
print(Score_list[no])
Ranking_probability_E

start : 2022.11.24 21:24:41
6,4


,1,2,3,4,Breakthrough
Spain,46.9%,37.0%,16.0%,0.1%,83.9%
CostaRica,0.0%,0.0%,27.4%,72.6%,0.0%
Germany,2.3%,21.7%,48.6%,27.3%,24.1%
Japan,51.0%,41.2%,7.8%,0.0%,92.2%


In [26]:
no = 17
Group_E_RoundRobin.iat[0, 1], Group_E_RoundRobin.iat[1, 0] = '7,0' , '0,7'
Group_E_RoundRobin.iat[2, 3], Group_E_RoundRobin.iat[3, 2] = '1,2' , '2,1'

#Group_E_RoundRobin.iat[0, 2], Group_E_RoundRobin.iat[2, 0] = '0,2' , '2,0'
Group_E_RoundRobin.iat[1, 3], Group_E_RoundRobin.iat[3, 1] = Score_inverselist[no] , Score_list[no]

#Group_E_RoundRobin.iat[0, 3], Group_E_RoundRobin.iat[3, 0] = '0,2' , '2,0'
#Group_E_RoundRobin.iat[1, 2], Group_E_RoundRobin.iat[2, 1] = '0,2' , '2,0'
Ranking(Group_E_RoundRobin)
Ranking_num_E, Ranking_probability_E = RankingProbability(Group_E_RoundRobin.copy())
for i in range(1,5):
    Ranking_JapanPropability.at[Score_list[no], i] = Ranking_probability_E.at['Japan', i]
Ranking_JapanPropability.at[Score_list[no], 'Breakthrough'] = Ranking_probability_E.at['Japan', 'Breakthrough']
print(Score_list[no])
Ranking_probability_E

start : 2022.11.24 21:30:41
5,3


,1,2,3,4,Breakthrough
Spain,46.9%,37.0%,16.0%,0.1%,83.9%
CostaRica,0.0%,0.0%,27.4%,72.6%,0.0%
Germany,2.3%,21.8%,48.7%,27.2%,24.1%
Japan,50.9%,41.2%,7.9%,0.0%,92.1%


In [27]:
no = 18
Group_E_RoundRobin.iat[0, 1], Group_E_RoundRobin.iat[1, 0] = '7,0' , '0,7'
Group_E_RoundRobin.iat[2, 3], Group_E_RoundRobin.iat[3, 2] = '1,2' , '2,1'

#Group_E_RoundRobin.iat[0, 2], Group_E_RoundRobin.iat[2, 0] = '0,2' , '2,0'
Group_E_RoundRobin.iat[1, 3], Group_E_RoundRobin.iat[3, 1] = Score_inverselist[no] , Score_list[no]

#Group_E_RoundRobin.iat[0, 3], Group_E_RoundRobin.iat[3, 0] = '0,2' , '2,0'
#Group_E_RoundRobin.iat[1, 2], Group_E_RoundRobin.iat[2, 1] = '0,2' , '2,0'
Ranking(Group_E_RoundRobin)
Ranking_num_E, Ranking_probability_E = RankingProbability(Group_E_RoundRobin.copy())
for i in range(1,5):
    Ranking_JapanPropability.at[Score_list[no], i] = Ranking_probability_E.at['Japan', i]
Ranking_JapanPropability.at[Score_list[no], 'Breakthrough'] = Ranking_probability_E.at['Japan', 'Breakthrough']
print(Score_list[no])
Ranking_probability_E

start : 2022.11.24 21:36:41
4,2


,1,2,3,4,Breakthrough
Spain,46.9%,37.0%,16.0%,0.1%,83.9%
CostaRica,0.0%,0.0%,27.3%,72.7%,0.0%
Germany,2.3%,21.8%,48.8%,27.1%,24.2%
Japan,50.9%,41.2%,7.9%,0.0%,92.1%


In [28]:
no = 19
Group_E_RoundRobin.iat[0, 1], Group_E_RoundRobin.iat[1, 0] = '7,0' , '0,7'
Group_E_RoundRobin.iat[2, 3], Group_E_RoundRobin.iat[3, 2] = '1,2' , '2,1'

#Group_E_RoundRobin.iat[0, 2], Group_E_RoundRobin.iat[2, 0] = '0,2' , '2,0'
Group_E_RoundRobin.iat[1, 3], Group_E_RoundRobin.iat[3, 1] = Score_inverselist[no] , Score_list[no]

#Group_E_RoundRobin.iat[0, 3], Group_E_RoundRobin.iat[3, 0] = '0,2' , '2,0'
#Group_E_RoundRobin.iat[1, 2], Group_E_RoundRobin.iat[2, 1] = '0,2' , '2,0'
Ranking(Group_E_RoundRobin)
Ranking_num_E, Ranking_probability_E = RankingProbability(Group_E_RoundRobin.copy())
for i in range(1,5):
    Ranking_JapanPropability.at[Score_list[no], i] = Ranking_probability_E.at['Japan', i]
Ranking_JapanPropability.at[Score_list[no], 'Breakthrough'] = Ranking_probability_E.at['Japan', 'Breakthrough']
print(Score_list[no])
Ranking_probability_E

start : 2022.11.24 21:42:39
3,1


,1,2,3,4,Breakthrough
Spain,46.9%,37.1%,16.0%,0.1%,83.9%
CostaRica,0.0%,0.0%,27.1%,72.8%,0.0%
Germany,2.3%,21.9%,48.9%,26.9%,24.2%
Japan,50.9%,41.1%,8.0%,0.0%,92.0%


In [29]:
no = 20
Group_E_RoundRobin.iat[0, 1], Group_E_RoundRobin.iat[1, 0] = '7,0' , '0,7'
Group_E_RoundRobin.iat[2, 3], Group_E_RoundRobin.iat[3, 2] = '1,2' , '2,1'

#Group_E_RoundRobin.iat[0, 2], Group_E_RoundRobin.iat[2, 0] = '0,2' , '2,0'
Group_E_RoundRobin.iat[1, 3], Group_E_RoundRobin.iat[3, 1] = Score_inverselist[no] , Score_list[no]

#Group_E_RoundRobin.iat[0, 3], Group_E_RoundRobin.iat[3, 0] = '0,2' , '2,0'
#Group_E_RoundRobin.iat[1, 2], Group_E_RoundRobin.iat[2, 1] = '0,2' , '2,0'
Ranking(Group_E_RoundRobin)
Ranking_num_E, Ranking_probability_E = RankingProbability(Group_E_RoundRobin.copy())
for i in range(1,5):
    Ranking_JapanPropability.at[Score_list[no], i] = Ranking_probability_E.at['Japan', i]
Ranking_JapanPropability.at[Score_list[no], 'Breakthrough'] = Ranking_probability_E.at['Japan', 'Breakthrough']
print(Score_list[no])
Ranking_probability_E

start : 2022.11.24 21:48:34
2,0


,1,2,3,4,Breakthrough
Spain,46.9%,37.1%,16.0%,0.1%,83.9%
CostaRica,0.0%,0.0%,27.0%,73.0%,0.0%
Germany,2.3%,21.9%,49.0%,26.8%,24.2%
Japan,50.9%,41.1%,8.0%,0.0%,92.0%


In [30]:
no = 21
Group_E_RoundRobin.iat[0, 1], Group_E_RoundRobin.iat[1, 0] = '7,0' , '0,7'
Group_E_RoundRobin.iat[2, 3], Group_E_RoundRobin.iat[3, 2] = '1,2' , '2,1'

#Group_E_RoundRobin.iat[0, 2], Group_E_RoundRobin.iat[2, 0] = '0,2' , '2,0'
Group_E_RoundRobin.iat[1, 3], Group_E_RoundRobin.iat[3, 1] = Score_inverselist[no] , Score_list[no]

#Group_E_RoundRobin.iat[0, 3], Group_E_RoundRobin.iat[3, 0] = '0,2' , '2,0'
#Group_E_RoundRobin.iat[1, 2], Group_E_RoundRobin.iat[2, 1] = '0,2' , '2,0'
Ranking(Group_E_RoundRobin)
Ranking_num_E, Ranking_probability_E = RankingProbability(Group_E_RoundRobin.copy())
for i in range(1,5):
    Ranking_JapanPropability.at[Score_list[no], i] = Ranking_probability_E.at['Japan', i]
Ranking_JapanPropability.at[Score_list[no], 'Breakthrough'] = Ranking_probability_E.at['Japan', 'Breakthrough']
print(Score_list[no])
Ranking_probability_E

start : 2022.11.24 21:54:34
7,6


,1,2,3,4,Breakthrough
Spain,46.9%,37.0%,15.9%,0.2%,83.8%
CostaRica,0.0%,0.1%,28.6%,71.3%,0.1%
Germany,2.3%,22.0%,47.2%,28.5%,24.3%
Japan,50.9%,41.0%,8.2%,0.0%,91.8%


In [31]:
no = 22
Group_E_RoundRobin.iat[0, 1], Group_E_RoundRobin.iat[1, 0] = '7,0' , '0,7'
Group_E_RoundRobin.iat[2, 3], Group_E_RoundRobin.iat[3, 2] = '1,2' , '2,1'

#Group_E_RoundRobin.iat[0, 2], Group_E_RoundRobin.iat[2, 0] = '0,2' , '2,0'
Group_E_RoundRobin.iat[1, 3], Group_E_RoundRobin.iat[3, 1] = Score_inverselist[no] , Score_list[no]

#Group_E_RoundRobin.iat[0, 3], Group_E_RoundRobin.iat[3, 0] = '0,2' , '2,0'
#Group_E_RoundRobin.iat[1, 2], Group_E_RoundRobin.iat[2, 1] = '0,2' , '2,0'
Ranking(Group_E_RoundRobin)
Ranking_num_E, Ranking_probability_E = RankingProbability(Group_E_RoundRobin.copy())
for i in range(1,5):
    Ranking_JapanPropability.at[Score_list[no], i] = Ranking_probability_E.at['Japan', i]
Ranking_JapanPropability.at[Score_list[no], 'Breakthrough'] = Ranking_probability_E.at['Japan', 'Breakthrough']
print(Score_list[no])
Ranking_probability_E

start : 2022.11.24 22:00:34
6,5


,1,2,3,4,Breakthrough
Spain,47.0%,37.0%,15.9%,0.2%,83.9%
CostaRica,0.0%,0.1%,28.6%,71.3%,0.1%
Germany,2.3%,22.0%,47.2%,28.5%,24.3%
Japan,50.9%,41.0%,8.2%,0.0%,91.8%


In [32]:
no = 23
Group_E_RoundRobin.iat[0, 1], Group_E_RoundRobin.iat[1, 0] = '7,0' , '0,7'
Group_E_RoundRobin.iat[2, 3], Group_E_RoundRobin.iat[3, 2] = '1,2' , '2,1'

#Group_E_RoundRobin.iat[0, 2], Group_E_RoundRobin.iat[2, 0] = '0,2' , '2,0'
Group_E_RoundRobin.iat[1, 3], Group_E_RoundRobin.iat[3, 1] = Score_inverselist[no] , Score_list[no]

#Group_E_RoundRobin.iat[0, 3], Group_E_RoundRobin.iat[3, 0] = '0,2' , '2,0'
#Group_E_RoundRobin.iat[1, 2], Group_E_RoundRobin.iat[2, 1] = '0,2' , '2,0'
Ranking(Group_E_RoundRobin)
Ranking_num_E, Ranking_probability_E = RankingProbability(Group_E_RoundRobin.copy())
for i in range(1,5):
    Ranking_JapanPropability.at[Score_list[no], i] = Ranking_probability_E.at['Japan', i]
Ranking_JapanPropability.at[Score_list[no], 'Breakthrough'] = Ranking_probability_E.at['Japan', 'Breakthrough']
print(Score_list[no])
Ranking_probability_E

start : 2022.11.24 22:06:28
5,4


,1,2,3,4,Breakthrough
Spain,47.0%,37.0%,15.9%,0.2%,83.9%
CostaRica,0.0%,0.1%,28.6%,71.3%,0.1%
Germany,2.3%,22.0%,47.2%,28.4%,24.4%
Japan,50.8%,41.0%,8.3%,0.0%,91.7%


In [33]:
no = 24
Group_E_RoundRobin.iat[0, 1], Group_E_RoundRobin.iat[1, 0] = '7,0' , '0,7'
Group_E_RoundRobin.iat[2, 3], Group_E_RoundRobin.iat[3, 2] = '1,2' , '2,1'

#Group_E_RoundRobin.iat[0, 2], Group_E_RoundRobin.iat[2, 0] = '0,2' , '2,0'
Group_E_RoundRobin.iat[1, 3], Group_E_RoundRobin.iat[3, 1] = Score_inverselist[no] , Score_list[no]

#Group_E_RoundRobin.iat[0, 3], Group_E_RoundRobin.iat[3, 0] = '0,2' , '2,0'
#Group_E_RoundRobin.iat[1, 2], Group_E_RoundRobin.iat[2, 1] = '0,2' , '2,0'
Ranking(Group_E_RoundRobin)
Ranking_num_E, Ranking_probability_E = RankingProbability(Group_E_RoundRobin.copy())
for i in range(1,5):
    Ranking_JapanPropability.at[Score_list[no], i] = Ranking_probability_E.at['Japan', i]
Ranking_JapanPropability.at[Score_list[no], 'Breakthrough'] = Ranking_probability_E.at['Japan', 'Breakthrough']
print(Score_list[no])
Ranking_probability_E

start : 2022.11.24 22:12:20
4,3


,1,2,3,4,Breakthrough
Spain,47.0%,37.0%,15.9%,0.2%,83.9%
CostaRica,0.0%,0.1%,28.5%,71.4%,0.1%
Germany,2.3%,22.0%,47.3%,28.3%,24.4%
Japan,50.8%,40.9%,8.3%,0.0%,91.7%


In [34]:
no = 25
Group_E_RoundRobin.iat[0, 1], Group_E_RoundRobin.iat[1, 0] = '7,0' , '0,7'
Group_E_RoundRobin.iat[2, 3], Group_E_RoundRobin.iat[3, 2] = '1,2' , '2,1'

#Group_E_RoundRobin.iat[0, 2], Group_E_RoundRobin.iat[2, 0] = '0,2' , '2,0'
Group_E_RoundRobin.iat[1, 3], Group_E_RoundRobin.iat[3, 1] = Score_inverselist[no] , Score_list[no]

#Group_E_RoundRobin.iat[0, 3], Group_E_RoundRobin.iat[3, 0] = '0,2' , '2,0'
#Group_E_RoundRobin.iat[1, 2], Group_E_RoundRobin.iat[2, 1] = '0,2' , '2,0'
Ranking(Group_E_RoundRobin)
Ranking_num_E, Ranking_probability_E = RankingProbability(Group_E_RoundRobin.copy())
for i in range(1,5):
    Ranking_JapanPropability.at[Score_list[no], i] = Ranking_probability_E.at['Japan', i]
Ranking_JapanPropability.at[Score_list[no], 'Breakthrough'] = Ranking_probability_E.at['Japan', 'Breakthrough']
print(Score_list[no])
Ranking_probability_E

start : 2022.11.24 22:18:14
3,2


,1,2,3,4,Breakthrough
Spain,47.0%,37.0%,15.9%,0.2%,84.0%
CostaRica,0.0%,0.1%,28.4%,71.5%,0.1%
Germany,2.3%,22.1%,47.5%,28.1%,24.4%
Japan,50.8%,40.9%,8.3%,0.0%,91.7%


In [35]:
no = 26
Group_E_RoundRobin.iat[0, 1], Group_E_RoundRobin.iat[1, 0] = '7,0' , '0,7'
Group_E_RoundRobin.iat[2, 3], Group_E_RoundRobin.iat[3, 2] = '1,2' , '2,1'

#Group_E_RoundRobin.iat[0, 2], Group_E_RoundRobin.iat[2, 0] = '0,2' , '2,0'
Group_E_RoundRobin.iat[1, 3], Group_E_RoundRobin.iat[3, 1] = Score_inverselist[no] , Score_list[no]

#Group_E_RoundRobin.iat[0, 3], Group_E_RoundRobin.iat[3, 0] = '0,2' , '2,0'
#Group_E_RoundRobin.iat[1, 2], Group_E_RoundRobin.iat[2, 1] = '0,2' , '2,0'
Ranking(Group_E_RoundRobin)
Ranking_num_E, Ranking_probability_E = RankingProbability(Group_E_RoundRobin.copy())
for i in range(1,5):
    Ranking_JapanPropability.at[Score_list[no], i] = Ranking_probability_E.at['Japan', i]
Ranking_JapanPropability.at[Score_list[no], 'Breakthrough'] = Ranking_probability_E.at['Japan', 'Breakthrough']
print(Score_list[no])
Ranking_probability_E

start : 2022.11.24 22:24:08
2,1


,1,2,3,4,Breakthrough
Spain,47.0%,37.0%,15.9%,0.1%,84.0%
CostaRica,0.0%,0.1%,28.2%,71.8%,0.1%
Germany,2.3%,22.1%,47.9%,27.7%,24.4%
Japan,50.8%,40.9%,8.3%,0.0%,91.7%


In [36]:
no = 27
Group_E_RoundRobin.iat[0, 1], Group_E_RoundRobin.iat[1, 0] = '7,0' , '0,7'
Group_E_RoundRobin.iat[2, 3], Group_E_RoundRobin.iat[3, 2] = '1,2' , '2,1'

#Group_E_RoundRobin.iat[0, 2], Group_E_RoundRobin.iat[2, 0] = '0,2' , '2,0'
Group_E_RoundRobin.iat[1, 3], Group_E_RoundRobin.iat[3, 1] = Score_inverselist[no] , Score_list[no]

#Group_E_RoundRobin.iat[0, 3], Group_E_RoundRobin.iat[3, 0] = '0,2' , '2,0'
#Group_E_RoundRobin.iat[1, 2], Group_E_RoundRobin.iat[2, 1] = '0,2' , '2,0'
Ranking(Group_E_RoundRobin)
Ranking_num_E, Ranking_probability_E = RankingProbability(Group_E_RoundRobin.copy())
for i in range(1,5):
    Ranking_JapanPropability.at[Score_list[no], i] = Ranking_probability_E.at['Japan', i]
Ranking_JapanPropability.at[Score_list[no], 'Breakthrough'] = Ranking_probability_E.at['Japan', 'Breakthrough']
print(Score_list[no])
Ranking_probability_E

start : 2022.11.24 22:30:03
1,0


,1,2,3,4,Breakthrough
Spain,47.0%,37.0%,15.9%,0.1%,84.0%
CostaRica,0.0%,0.1%,27.8%,72.2%,0.1%
Germany,2.3%,22.1%,48.1%,27.5%,24.4%
Japan,50.8%,40.9%,8.3%,0.0%,91.7%


In [37]:
no = 28
Group_E_RoundRobin.iat[0, 1], Group_E_RoundRobin.iat[1, 0] = '7,0' , '0,7'
Group_E_RoundRobin.iat[2, 3], Group_E_RoundRobin.iat[3, 2] = '1,2' , '2,1'

#Group_E_RoundRobin.iat[0, 2], Group_E_RoundRobin.iat[2, 0] = '0,2' , '2,0'
Group_E_RoundRobin.iat[1, 3], Group_E_RoundRobin.iat[3, 1] = Score_inverselist[no] , Score_list[no]

#Group_E_RoundRobin.iat[0, 3], Group_E_RoundRobin.iat[3, 0] = '0,2' , '2,0'
#Group_E_RoundRobin.iat[1, 2], Group_E_RoundRobin.iat[2, 1] = '0,2' , '2,0'
Ranking(Group_E_RoundRobin)
Ranking_num_E, Ranking_probability_E = RankingProbability(Group_E_RoundRobin.copy())
for i in range(1,5):
    Ranking_JapanPropability.at[Score_list[no], i] = Ranking_probability_E.at['Japan', i]
Ranking_JapanPropability.at[Score_list[no], 'Breakthrough'] = Ranking_probability_E.at['Japan', 'Breakthrough']
print(Score_list[no])
Ranking_probability_E

start : 2022.11.24 22:35:58
7,7


,1,2,3,4,Breakthrough
Spain,48.5%,29.4%,19.8%,2.3%,77.9%
CostaRica,0.0%,13.1%,36.1%,50.8%,13.1%
Germany,4.7%,21.9%,26.5%,46.9%,26.6%
Japan,46.8%,35.9%,17.3%,0.0%,82.7%


In [38]:
no = 29
Group_E_RoundRobin.iat[0, 1], Group_E_RoundRobin.iat[1, 0] = '7,0' , '0,7'
Group_E_RoundRobin.iat[2, 3], Group_E_RoundRobin.iat[3, 2] = '1,2' , '2,1'

#Group_E_RoundRobin.iat[0, 2], Group_E_RoundRobin.iat[2, 0] = '0,2' , '2,0'
Group_E_RoundRobin.iat[1, 3], Group_E_RoundRobin.iat[3, 1] = Score_inverselist[no] , Score_list[no]

#Group_E_RoundRobin.iat[0, 3], Group_E_RoundRobin.iat[3, 0] = '0,2' , '2,0'
#Group_E_RoundRobin.iat[1, 2], Group_E_RoundRobin.iat[2, 1] = '0,2' , '2,0'
Ranking(Group_E_RoundRobin)
Ranking_num_E, Ranking_probability_E = RankingProbability(Group_E_RoundRobin.copy())
for i in range(1,5):
    Ranking_JapanPropability.at[Score_list[no], i] = Ranking_probability_E.at['Japan', i]
Ranking_JapanPropability.at[Score_list[no], 'Breakthrough'] = Ranking_probability_E.at['Japan', 'Breakthrough']
print(Score_list[no])
Ranking_probability_E

start : 2022.11.24 22:41:53
6,6


,1,2,3,4,Breakthrough
Spain,48.5%,29.4%,19.8%,2.3%,77.9%
CostaRica,0.0%,13.1%,36.1%,50.8%,13.1%
Germany,4.7%,21.9%,26.5%,46.9%,26.6%
Japan,46.8%,35.9%,17.3%,0.0%,82.7%


In [39]:
no = 30
Group_E_RoundRobin.iat[0, 1], Group_E_RoundRobin.iat[1, 0] = '7,0' , '0,7'
Group_E_RoundRobin.iat[2, 3], Group_E_RoundRobin.iat[3, 2] = '1,2' , '2,1'

#Group_E_RoundRobin.iat[0, 2], Group_E_RoundRobin.iat[2, 0] = '0,2' , '2,0'
Group_E_RoundRobin.iat[1, 3], Group_E_RoundRobin.iat[3, 1] = Score_inverselist[no] , Score_list[no]

#Group_E_RoundRobin.iat[0, 3], Group_E_RoundRobin.iat[3, 0] = '0,2' , '2,0'
#Group_E_RoundRobin.iat[1, 2], Group_E_RoundRobin.iat[2, 1] = '0,2' , '2,0'
Ranking(Group_E_RoundRobin)
Ranking_num_E, Ranking_probability_E = RankingProbability(Group_E_RoundRobin.copy())
for i in range(1,5):
    Ranking_JapanPropability.at[Score_list[no], i] = Ranking_probability_E.at['Japan', i]
Ranking_JapanPropability.at[Score_list[no], 'Breakthrough'] = Ranking_probability_E.at['Japan', 'Breakthrough']
print(Score_list[no])
Ranking_probability_E

start : 2022.11.24 22:47:48
5,5


,1,2,3,4,Breakthrough
Spain,48.5%,29.4%,19.8%,2.3%,77.9%
CostaRica,0.0%,13.1%,36.1%,50.8%,13.1%
Germany,4.7%,22.0%,26.5%,46.9%,26.7%
Japan,46.8%,35.8%,17.3%,0.0%,82.7%


In [40]:
no = 31
Group_E_RoundRobin.iat[0, 1], Group_E_RoundRobin.iat[1, 0] = '7,0' , '0,7'
Group_E_RoundRobin.iat[2, 3], Group_E_RoundRobin.iat[3, 2] = '1,2' , '2,1'

#Group_E_RoundRobin.iat[0, 2], Group_E_RoundRobin.iat[2, 0] = '0,2' , '2,0'
Group_E_RoundRobin.iat[1, 3], Group_E_RoundRobin.iat[3, 1] = Score_inverselist[no] , Score_list[no]

#Group_E_RoundRobin.iat[0, 3], Group_E_RoundRobin.iat[3, 0] = '0,2' , '2,0'
#Group_E_RoundRobin.iat[1, 2], Group_E_RoundRobin.iat[2, 1] = '0,2' , '2,0'
Ranking(Group_E_RoundRobin)
Ranking_num_E, Ranking_probability_E = RankingProbability(Group_E_RoundRobin.copy())
for i in range(1,5):
    Ranking_JapanPropability.at[Score_list[no], i] = Ranking_probability_E.at['Japan', i]
Ranking_JapanPropability.at[Score_list[no], 'Breakthrough'] = Ranking_probability_E.at['Japan', 'Breakthrough']
print(Score_list[no])
Ranking_probability_E

start : 2022.11.24 22:53:44
4,4


,1,2,3,4,Breakthrough
Spain,48.5%,29.4%,19.8%,2.3%,77.9%
CostaRica,0.0%,13.1%,36.1%,50.8%,13.1%
Germany,4.7%,22.0%,26.4%,46.9%,26.7%
Japan,46.8%,35.8%,17.4%,0.0%,82.6%


In [41]:
no = 32
Group_E_RoundRobin.iat[0, 1], Group_E_RoundRobin.iat[1, 0] = '7,0' , '0,7'
Group_E_RoundRobin.iat[2, 3], Group_E_RoundRobin.iat[3, 2] = '1,2' , '2,1'

#Group_E_RoundRobin.iat[0, 2], Group_E_RoundRobin.iat[2, 0] = '0,2' , '2,0'
Group_E_RoundRobin.iat[1, 3], Group_E_RoundRobin.iat[3, 1] = Score_inverselist[no] , Score_list[no]

#Group_E_RoundRobin.iat[0, 3], Group_E_RoundRobin.iat[3, 0] = '0,2' , '2,0'
#Group_E_RoundRobin.iat[1, 2], Group_E_RoundRobin.iat[2, 1] = '0,2' , '2,0'
Ranking(Group_E_RoundRobin)
Ranking_num_E, Ranking_probability_E = RankingProbability(Group_E_RoundRobin.copy())
for i in range(1,5):
    Ranking_JapanPropability.at[Score_list[no], i] = Ranking_probability_E.at['Japan', i]
Ranking_JapanPropability.at[Score_list[no], 'Breakthrough'] = Ranking_probability_E.at['Japan', 'Breakthrough']
print(Score_list[no])
Ranking_probability_E

start : 2022.11.24 22:59:38
3,3


,1,2,3,4,Breakthrough
Spain,48.5%,29.4%,19.8%,2.3%,77.9%
CostaRica,0.0%,13.1%,36.1%,50.8%,13.1%
Germany,4.7%,22.0%,26.4%,46.9%,26.7%
Japan,46.8%,35.8%,17.4%,0.0%,82.6%


In [42]:
no = 33
Group_E_RoundRobin.iat[0, 1], Group_E_RoundRobin.iat[1, 0] = '7,0' , '0,7'
Group_E_RoundRobin.iat[2, 3], Group_E_RoundRobin.iat[3, 2] = '1,2' , '2,1'

#Group_E_RoundRobin.iat[0, 2], Group_E_RoundRobin.iat[2, 0] = '0,2' , '2,0'
Group_E_RoundRobin.iat[1, 3], Group_E_RoundRobin.iat[3, 1] = Score_inverselist[no] , Score_list[no]

#Group_E_RoundRobin.iat[0, 3], Group_E_RoundRobin.iat[3, 0] = '0,2' , '2,0'
#Group_E_RoundRobin.iat[1, 2], Group_E_RoundRobin.iat[2, 1] = '0,2' , '2,0'
Ranking(Group_E_RoundRobin)
Ranking_num_E, Ranking_probability_E = RankingProbability(Group_E_RoundRobin.copy())
for i in range(1,5):
    Ranking_JapanPropability.at[Score_list[no], i] = Ranking_probability_E.at['Japan', i]
Ranking_JapanPropability.at[Score_list[no], 'Breakthrough'] = Ranking_probability_E.at['Japan', 'Breakthrough']
print(Score_list[no])
Ranking_probability_E

start : 2022.11.24 23:05:35
2,2


,1,2,3,4,Breakthrough
Spain,48.5%,29.4%,19.8%,2.3%,77.9%
CostaRica,0.0%,13.1%,36.1%,50.8%,13.1%
Germany,4.7%,22.1%,26.4%,46.9%,26.8%
Japan,46.8%,35.7%,17.4%,0.0%,82.6%


In [43]:
no = 34
Group_E_RoundRobin.iat[0, 1], Group_E_RoundRobin.iat[1, 0] = '7,0' , '0,7'
Group_E_RoundRobin.iat[2, 3], Group_E_RoundRobin.iat[3, 2] = '1,2' , '2,1'

#Group_E_RoundRobin.iat[0, 2], Group_E_RoundRobin.iat[2, 0] = '0,2' , '2,0'
Group_E_RoundRobin.iat[1, 3], Group_E_RoundRobin.iat[3, 1] = Score_inverselist[no] , Score_list[no]

#Group_E_RoundRobin.iat[0, 3], Group_E_RoundRobin.iat[3, 0] = '0,2' , '2,0'
#Group_E_RoundRobin.iat[1, 2], Group_E_RoundRobin.iat[2, 1] = '0,2' , '2,0'
Ranking(Group_E_RoundRobin)
Ranking_num_E, Ranking_probability_E = RankingProbability(Group_E_RoundRobin.copy())
for i in range(1,5):
    Ranking_JapanPropability.at[Score_list[no], i] = Ranking_probability_E.at['Japan', i]
Ranking_JapanPropability.at[Score_list[no], 'Breakthrough'] = Ranking_probability_E.at['Japan', 'Breakthrough']
print(Score_list[no])
Ranking_probability_E

start : 2022.11.24 23:11:30
1,1


,1,2,3,4,Breakthrough
Spain,48.5%,29.4%,19.8%,2.3%,77.9%
CostaRica,0.0%,13.1%,36.1%,50.8%,13.1%
Germany,4.7%,22.1%,26.3%,46.9%,26.8%
Japan,46.8%,35.7%,17.5%,0.0%,82.5%


In [44]:
no = 35
Group_E_RoundRobin.iat[0, 1], Group_E_RoundRobin.iat[1, 0] = '7,0' , '0,7'
Group_E_RoundRobin.iat[2, 3], Group_E_RoundRobin.iat[3, 2] = '1,2' , '2,1'

#Group_E_RoundRobin.iat[0, 2], Group_E_RoundRobin.iat[2, 0] = '0,2' , '2,0'
Group_E_RoundRobin.iat[1, 3], Group_E_RoundRobin.iat[3, 1] = Score_inverselist[no] , Score_list[no]

#Group_E_RoundRobin.iat[0, 3], Group_E_RoundRobin.iat[3, 0] = '0,2' , '2,0'
#Group_E_RoundRobin.iat[1, 2], Group_E_RoundRobin.iat[2, 1] = '0,2' , '2,0'
Ranking(Group_E_RoundRobin)
Ranking_num_E, Ranking_probability_E = RankingProbability(Group_E_RoundRobin.copy())
for i in range(1,5):
    Ranking_JapanPropability.at[Score_list[no], i] = Ranking_probability_E.at['Japan', i]
Ranking_JapanPropability.at[Score_list[no], 'Breakthrough'] = Ranking_probability_E.at['Japan', 'Breakthrough']
print(Score_list[no])
Ranking_probability_E

start : 2022.11.24 23:17:23
0,0


,1,2,3,4,Breakthrough
Spain,48.5%,29.4%,19.8%,2.3%,77.9%
CostaRica,0.0%,13.1%,36.1%,50.8%,13.1%
Germany,4.7%,22.1%,26.3%,46.9%,26.8%
Japan,46.8%,35.7%,17.5%,0.0%,82.5%


In [45]:
no = 36
Group_E_RoundRobin.iat[0, 1], Group_E_RoundRobin.iat[1, 0] = '7,0' , '0,7'
Group_E_RoundRobin.iat[2, 3], Group_E_RoundRobin.iat[3, 2] = '1,2' , '2,1'

#Group_E_RoundRobin.iat[0, 2], Group_E_RoundRobin.iat[2, 0] = '0,2' , '2,0'
Group_E_RoundRobin.iat[1, 3], Group_E_RoundRobin.iat[3, 1] = Score_inverselist[no] , Score_list[no]

#Group_E_RoundRobin.iat[0, 3], Group_E_RoundRobin.iat[3, 0] = '0,2' , '2,0'
#Group_E_RoundRobin.iat[1, 2], Group_E_RoundRobin.iat[2, 1] = '0,2' , '2,0'
Ranking(Group_E_RoundRobin)
Ranking_num_E, Ranking_probability_E = RankingProbability(Group_E_RoundRobin.copy())
for i in range(1,5):
    Ranking_JapanPropability.at[Score_list[no], i] = Ranking_probability_E.at['Japan', i]
Ranking_JapanPropability.at[Score_list[no], 'Breakthrough'] = Ranking_probability_E.at['Japan', 'Breakthrough']
print(Score_list[no])
Ranking_probability_E

start : 2022.11.24 23:23:18
6,7


,1,2,3,4,Breakthrough
Spain,64.2%,13.7%,17.4%,4.7%,77.9%
CostaRica,3.2%,35.4%,17.4%,44.1%,38.5%
Germany,10.8%,23.0%,23.5%,42.7%,33.7%
Japan,22.2%,27.8%,41.6%,8.3%,50.0%


In [46]:
no = 37
Group_E_RoundRobin.iat[0, 1], Group_E_RoundRobin.iat[1, 0] = '7,0' , '0,7'
Group_E_RoundRobin.iat[2, 3], Group_E_RoundRobin.iat[3, 2] = '1,2' , '2,1'

#Group_E_RoundRobin.iat[0, 2], Group_E_RoundRobin.iat[2, 0] = '0,2' , '2,0'
Group_E_RoundRobin.iat[1, 3], Group_E_RoundRobin.iat[3, 1] = Score_inverselist[no] , Score_list[no]

#Group_E_RoundRobin.iat[0, 3], Group_E_RoundRobin.iat[3, 0] = '0,2' , '2,0'
#Group_E_RoundRobin.iat[1, 2], Group_E_RoundRobin.iat[2, 1] = '0,2' , '2,0'
Ranking(Group_E_RoundRobin)
Ranking_num_E, Ranking_probability_E = RankingProbability(Group_E_RoundRobin.copy())
for i in range(1,5):
    Ranking_JapanPropability.at[Score_list[no], i] = Ranking_probability_E.at['Japan', i]
Ranking_JapanPropability.at[Score_list[no], 'Breakthrough'] = Ranking_probability_E.at['Japan', 'Breakthrough']
print(Score_list[no])
Ranking_probability_E

start : 2022.11.24 23:30:38
5,6


,1,2,3,4,Breakthrough
Spain,64.4%,13.7%,17.2%,4.7%,78.1%
CostaRica,3.2%,35.4%,17.4%,44.1%,38.5%
Germany,10.9%,23.0%,23.5%,42.6%,33.9%
Japan,22.0%,27.9%,41.7%,8.4%,49.9%


In [47]:
no = 38
Group_E_RoundRobin.iat[0, 1], Group_E_RoundRobin.iat[1, 0] = '7,0' , '0,7'
Group_E_RoundRobin.iat[2, 3], Group_E_RoundRobin.iat[3, 2] = '1,2' , '2,1'

#Group_E_RoundRobin.iat[0, 2], Group_E_RoundRobin.iat[2, 0] = '0,2' , '2,0'
Group_E_RoundRobin.iat[1, 3], Group_E_RoundRobin.iat[3, 1] = Score_inverselist[no] , Score_list[no]

#Group_E_RoundRobin.iat[0, 3], Group_E_RoundRobin.iat[3, 0] = '0,2' , '2,0'
#Group_E_RoundRobin.iat[1, 2], Group_E_RoundRobin.iat[2, 1] = '0,2' , '2,0'
Ranking(Group_E_RoundRobin)
Ranking_num_E, Ranking_probability_E = RankingProbability(Group_E_RoundRobin.copy())
for i in range(1,5):
    Ranking_JapanPropability.at[Score_list[no], i] = Ranking_probability_E.at['Japan', i]
Ranking_JapanPropability.at[Score_list[no], 'Breakthrough'] = Ranking_probability_E.at['Japan', 'Breakthrough']
print(Score_list[no])
Ranking_probability_E

start : 2022.11.24 23:37:57
4,5


,1,2,3,4,Breakthrough
Spain,64.6%,13.5%,17.2%,4.7%,78.1%
CostaRica,3.2%,35.4%,17.4%,44.1%,38.5%
Germany,11.0%,23.0%,23.5%,42.5%,34.0%
Japan,21.6%,27.9%,41.9%,8.5%,49.6%


In [48]:
no = 39
Group_E_RoundRobin.iat[0, 1], Group_E_RoundRobin.iat[1, 0] = '7,0' , '0,7'
Group_E_RoundRobin.iat[2, 3], Group_E_RoundRobin.iat[3, 2] = '1,2' , '2,1'

#Group_E_RoundRobin.iat[0, 2], Group_E_RoundRobin.iat[2, 0] = '0,2' , '2,0'
Group_E_RoundRobin.iat[1, 3], Group_E_RoundRobin.iat[3, 1] = Score_inverselist[no] , Score_list[no]

#Group_E_RoundRobin.iat[0, 3], Group_E_RoundRobin.iat[3, 0] = '0,2' , '2,0'
#Group_E_RoundRobin.iat[1, 2], Group_E_RoundRobin.iat[2, 1] = '0,2' , '2,0'
Ranking(Group_E_RoundRobin)
Ranking_num_E, Ranking_probability_E = RankingProbability(Group_E_RoundRobin.copy())
for i in range(1,5):
    Ranking_JapanPropability.at[Score_list[no], i] = Ranking_probability_E.at['Japan', i]
Ranking_JapanPropability.at[Score_list[no], 'Breakthrough'] = Ranking_probability_E.at['Japan', 'Breakthrough']
print(Score_list[no])
Ranking_probability_E

start : 2022.11.24 23:45:16
3,4


,1,2,3,4,Breakthrough
Spain,64.7%,13.3%,17.2%,4.7%,78.1%
CostaRica,3.2%,35.4%,17.4%,44.1%,38.5%
Germany,11.1%,23.0%,23.5%,42.4%,34.2%
Japan,21.3%,28.1%,41.9%,8.6%,49.4%


In [49]:
no = 40
Group_E_RoundRobin.iat[0, 1], Group_E_RoundRobin.iat[1, 0] = '7,0' , '0,7'
Group_E_RoundRobin.iat[2, 3], Group_E_RoundRobin.iat[3, 2] = '1,2' , '2,1'

#Group_E_RoundRobin.iat[0, 2], Group_E_RoundRobin.iat[2, 0] = '0,2' , '2,0'
Group_E_RoundRobin.iat[1, 3], Group_E_RoundRobin.iat[3, 1] = Score_inverselist[no] , Score_list[no]

#Group_E_RoundRobin.iat[0, 3], Group_E_RoundRobin.iat[3, 0] = '0,2' , '2,0'
#Group_E_RoundRobin.iat[1, 2], Group_E_RoundRobin.iat[2, 1] = '0,2' , '2,0'
Ranking(Group_E_RoundRobin)
Ranking_num_E, Ranking_probability_E = RankingProbability(Group_E_RoundRobin.copy())
for i in range(1,5):
    Ranking_JapanPropability.at[Score_list[no], i] = Ranking_probability_E.at['Japan', i]
Ranking_JapanPropability.at[Score_list[no], 'Breakthrough'] = Ranking_probability_E.at['Japan', 'Breakthrough']
print(Score_list[no])
Ranking_probability_E

start : 2022.11.24 23:52:37
2,3


,1,2,3,4,Breakthrough
Spain,64.8%,13.3%,17.2%,4.7%,78.1%
CostaRica,3.2%,35.4%,17.4%,44.1%,38.5%
Germany,11.3%,23.1%,23.4%,42.3%,34.3%
Japan,21.0%,28.2%,42.0%,8.8%,49.3%


In [50]:
no = 41
Group_E_RoundRobin.iat[0, 1], Group_E_RoundRobin.iat[1, 0] = '7,0' , '0,7'
Group_E_RoundRobin.iat[2, 3], Group_E_RoundRobin.iat[3, 2] = '1,2' , '2,1'

#Group_E_RoundRobin.iat[0, 2], Group_E_RoundRobin.iat[2, 0] = '0,2' , '2,0'
Group_E_RoundRobin.iat[1, 3], Group_E_RoundRobin.iat[3, 1] = Score_inverselist[no] , Score_list[no]

#Group_E_RoundRobin.iat[0, 3], Group_E_RoundRobin.iat[3, 0] = '0,2' , '2,0'
#Group_E_RoundRobin.iat[1, 2], Group_E_RoundRobin.iat[2, 1] = '0,2' , '2,0'
Ranking(Group_E_RoundRobin)
Ranking_num_E, Ranking_probability_E = RankingProbability(Group_E_RoundRobin.copy())
for i in range(1,5):
    Ranking_JapanPropability.at[Score_list[no], i] = Ranking_probability_E.at['Japan', i]
Ranking_JapanPropability.at[Score_list[no], 'Breakthrough'] = Ranking_probability_E.at['Japan', 'Breakthrough']
print(Score_list[no])
Ranking_probability_E

start : 2022.11.24 23:59:55
1,2


,1,2,3,4,Breakthrough
Spain,64.8%,13.3%,17.2%,4.7%,78.1%
CostaRica,3.2%,35.4%,17.4%,44.1%,38.5%
Germany,11.4%,23.1%,23.6%,42.0%,34.5%
Japan,20.8%,28.3%,42.0%,8.9%,49.1%


In [51]:
no = 42
Group_E_RoundRobin.iat[0, 1], Group_E_RoundRobin.iat[1, 0] = '7,0' , '0,7'
Group_E_RoundRobin.iat[2, 3], Group_E_RoundRobin.iat[3, 2] = '1,2' , '2,1'

#Group_E_RoundRobin.iat[0, 2], Group_E_RoundRobin.iat[2, 0] = '0,2' , '2,0'
Group_E_RoundRobin.iat[1, 3], Group_E_RoundRobin.iat[3, 1] = Score_inverselist[no] , Score_list[no]

#Group_E_RoundRobin.iat[0, 3], Group_E_RoundRobin.iat[3, 0] = '0,2' , '2,0'
#Group_E_RoundRobin.iat[1, 2], Group_E_RoundRobin.iat[2, 1] = '0,2' , '2,0'
Ranking(Group_E_RoundRobin)
Ranking_num_E, Ranking_probability_E = RankingProbability(Group_E_RoundRobin.copy())
for i in range(1,5):
    Ranking_JapanPropability.at[Score_list[no], i] = Ranking_probability_E.at['Japan', i]
Ranking_JapanPropability.at[Score_list[no], 'Breakthrough'] = Ranking_probability_E.at['Japan', 'Breakthrough']
print(Score_list[no])
Ranking_probability_E

start : 2022.11.25 00:07:23
0,1


,1,2,3,4,Breakthrough
Spain,64.8%,13.3%,17.2%,4.7%,78.1%
CostaRica,3.2%,35.4%,17.2%,44.3%,38.5%
Germany,11.5%,23.1%,23.5%,41.9%,34.6%
Japan,20.7%,28.3%,42.0%,9.0%,49.0%


In [52]:
no = 43
Group_E_RoundRobin.iat[0, 1], Group_E_RoundRobin.iat[1, 0] = '7,0' , '0,7'
Group_E_RoundRobin.iat[2, 3], Group_E_RoundRobin.iat[3, 2] = '1,2' , '2,1'

#Group_E_RoundRobin.iat[0, 2], Group_E_RoundRobin.iat[2, 0] = '0,2' , '2,0'
Group_E_RoundRobin.iat[1, 3], Group_E_RoundRobin.iat[3, 1] = Score_inverselist[no] , Score_list[no]

#Group_E_RoundRobin.iat[0, 3], Group_E_RoundRobin.iat[3, 0] = '0,2' , '2,0'
#Group_E_RoundRobin.iat[1, 2], Group_E_RoundRobin.iat[2, 1] = '0,2' , '2,0'
Ranking(Group_E_RoundRobin)
Ranking_num_E, Ranking_probability_E = RankingProbability(Group_E_RoundRobin.copy())
for i in range(1,5):
    Ranking_JapanPropability.at[Score_list[no], i] = Ranking_probability_E.at['Japan', i]
Ranking_JapanPropability.at[Score_list[no], 'Breakthrough'] = Ranking_probability_E.at['Japan', 'Breakthrough']
print(Score_list[no])
Ranking_probability_E

start : 2022.11.25 00:14:55
5,7


,1,2,3,4,Breakthrough
Spain,65.1%,12.9%,17.2%,4.7%,78.1%
CostaRica,3.9%,35.2%,18.5%,42.3%,39.2%
Germany,11.7%,23.2%,23.0%,42.1%,34.9%
Japan,19.6%,28.5%,41.3%,10.6%,48.1%


In [53]:
no = 44
Group_E_RoundRobin.iat[0, 1], Group_E_RoundRobin.iat[1, 0] = '7,0' , '0,7'
Group_E_RoundRobin.iat[2, 3], Group_E_RoundRobin.iat[3, 2] = '1,2' , '2,1'

#Group_E_RoundRobin.iat[0, 2], Group_E_RoundRobin.iat[2, 0] = '0,2' , '2,0'
Group_E_RoundRobin.iat[1, 3], Group_E_RoundRobin.iat[3, 1] = Score_inverselist[no] , Score_list[no]

#Group_E_RoundRobin.iat[0, 3], Group_E_RoundRobin.iat[3, 0] = '0,2' , '2,0'
#Group_E_RoundRobin.iat[1, 2], Group_E_RoundRobin.iat[2, 1] = '0,2' , '2,0'
Ranking(Group_E_RoundRobin)
Ranking_num_E, Ranking_probability_E = RankingProbability(Group_E_RoundRobin.copy())
for i in range(1,5):
    Ranking_JapanPropability.at[Score_list[no], i] = Ranking_probability_E.at['Japan', i]
Ranking_JapanPropability.at[Score_list[no], 'Breakthrough'] = Ranking_probability_E.at['Japan', 'Breakthrough']
print(Score_list[no])
Ranking_probability_E

start : 2022.11.25 00:22:26
4,6


,1,2,3,4,Breakthrough
Spain,65.3%,12.8%,17.2%,4.7%,78.1%
CostaRica,3.9%,35.2%,18.5%,42.3%,39.2%
Germany,11.8%,23.2%,23.0%,42.0%,35.0%
Japan,19.4%,28.6%,41.3%,10.7%,48.0%


In [54]:
no = 45
Group_E_RoundRobin.iat[0, 1], Group_E_RoundRobin.iat[1, 0] = '7,0' , '0,7'
Group_E_RoundRobin.iat[2, 3], Group_E_RoundRobin.iat[3, 2] = '1,2' , '2,1'

#Group_E_RoundRobin.iat[0, 2], Group_E_RoundRobin.iat[2, 0] = '0,2' , '2,0'
Group_E_RoundRobin.iat[1, 3], Group_E_RoundRobin.iat[3, 1] = Score_inverselist[no] , Score_list[no]

#Group_E_RoundRobin.iat[0, 3], Group_E_RoundRobin.iat[3, 0] = '0,2' , '2,0'
#Group_E_RoundRobin.iat[1, 2], Group_E_RoundRobin.iat[2, 1] = '0,2' , '2,0'
Ranking(Group_E_RoundRobin)
Ranking_num_E, Ranking_probability_E = RankingProbability(Group_E_RoundRobin.copy())
for i in range(1,5):
    Ranking_JapanPropability.at[Score_list[no], i] = Ranking_probability_E.at['Japan', i]
Ranking_JapanPropability.at[Score_list[no], 'Breakthrough'] = Ranking_probability_E.at['Japan', 'Breakthrough']
print(Score_list[no])
Ranking_probability_E

start : 2022.11.25 00:29:50
3,5


,1,2,3,4,Breakthrough
Spain,65.4%,12.7%,17.2%,4.7%,78.1%
CostaRica,3.9%,35.2%,18.5%,42.3%,39.2%
Germany,11.9%,23.2%,23.0%,41.9%,35.1%
Japan,19.1%,28.8%,41.3%,10.7%,47.9%


In [55]:
no = 46
Group_E_RoundRobin.iat[0, 1], Group_E_RoundRobin.iat[1, 0] = '7,0' , '0,7'
Group_E_RoundRobin.iat[2, 3], Group_E_RoundRobin.iat[3, 2] = '1,2' , '2,1'

#Group_E_RoundRobin.iat[0, 2], Group_E_RoundRobin.iat[2, 0] = '0,2' , '2,0'
Group_E_RoundRobin.iat[1, 3], Group_E_RoundRobin.iat[3, 1] = Score_inverselist[no] , Score_list[no]

#Group_E_RoundRobin.iat[0, 3], Group_E_RoundRobin.iat[3, 0] = '0,2' , '2,0'
#Group_E_RoundRobin.iat[1, 2], Group_E_RoundRobin.iat[2, 1] = '0,2' , '2,0'
Ranking(Group_E_RoundRobin)
Ranking_num_E, Ranking_probability_E = RankingProbability(Group_E_RoundRobin.copy())
for i in range(1,5):
    Ranking_JapanPropability.at[Score_list[no], i] = Ranking_probability_E.at['Japan', i]
Ranking_JapanPropability.at[Score_list[no], 'Breakthrough'] = Ranking_probability_E.at['Japan', 'Breakthrough']
print(Score_list[no])
Ranking_probability_E

start : 2022.11.25 00:37:14
2,4


,1,2,3,4,Breakthrough
Spain,65.5%,12.6%,17.2%,4.7%,78.1%
CostaRica,3.9%,35.2%,18.5%,42.3%,39.2%
Germany,12.0%,23.1%,23.0%,41.8%,35.1%
Japan,18.9%,28.9%,41.3%,10.8%,47.8%


In [56]:
no = 47
Group_E_RoundRobin.iat[0, 1], Group_E_RoundRobin.iat[1, 0] = '7,0' , '0,7'
Group_E_RoundRobin.iat[2, 3], Group_E_RoundRobin.iat[3, 2] = '1,2' , '2,1'

#Group_E_RoundRobin.iat[0, 2], Group_E_RoundRobin.iat[2, 0] = '0,2' , '2,0'
Group_E_RoundRobin.iat[1, 3], Group_E_RoundRobin.iat[3, 1] = Score_inverselist[no] , Score_list[no]

#Group_E_RoundRobin.iat[0, 3], Group_E_RoundRobin.iat[3, 0] = '0,2' , '2,0'
#Group_E_RoundRobin.iat[1, 2], Group_E_RoundRobin.iat[2, 1] = '0,2' , '2,0'
Ranking(Group_E_RoundRobin)
Ranking_num_E, Ranking_probability_E = RankingProbability(Group_E_RoundRobin.copy())
for i in range(1,5):
    Ranking_JapanPropability.at[Score_list[no], i] = Ranking_probability_E.at['Japan', i]
Ranking_JapanPropability.at[Score_list[no], 'Breakthrough'] = Ranking_probability_E.at['Japan', 'Breakthrough']
print(Score_list[no])
Ranking_probability_E

start : 2022.11.25 00:44:38
1,3


,1,2,3,4,Breakthrough
Spain,65.5%,12.6%,17.2%,4.7%,78.1%
CostaRica,3.9%,35.2%,18.5%,42.3%,39.2%
Germany,12.1%,23.2%,23.2%,41.6%,35.2%
Japan,18.8%,29.0%,41.3%,10.9%,47.7%


In [57]:
no = 48
Group_E_RoundRobin.iat[0, 1], Group_E_RoundRobin.iat[1, 0] = '7,0' , '0,7'
Group_E_RoundRobin.iat[2, 3], Group_E_RoundRobin.iat[3, 2] = '1,2' , '2,1'

#Group_E_RoundRobin.iat[0, 2], Group_E_RoundRobin.iat[2, 0] = '0,2' , '2,0'
Group_E_RoundRobin.iat[1, 3], Group_E_RoundRobin.iat[3, 1] = Score_inverselist[no] , Score_list[no]

#Group_E_RoundRobin.iat[0, 3], Group_E_RoundRobin.iat[3, 0] = '0,2' , '2,0'
#Group_E_RoundRobin.iat[1, 2], Group_E_RoundRobin.iat[2, 1] = '0,2' , '2,0'
Ranking(Group_E_RoundRobin)
Ranking_num_E, Ranking_probability_E = RankingProbability(Group_E_RoundRobin.copy())
for i in range(1,5):
    Ranking_JapanPropability.at[Score_list[no], i] = Ranking_probability_E.at['Japan', i]
Ranking_JapanPropability.at[Score_list[no], 'Breakthrough'] = Ranking_probability_E.at['Japan', 'Breakthrough']
print(Score_list[no])
Ranking_probability_E

start : 2022.11.25 00:52:01
0,2


,1,2,3,4,Breakthrough
Spain,65.5%,12.6%,17.2%,4.7%,78.1%
CostaRica,3.9%,35.2%,18.4%,42.5%,39.1%
Germany,12.2%,23.2%,23.3%,41.4%,35.3%
Japan,18.7%,29.0%,41.3%,11.0%,47.7%


In [58]:
no = 49
Group_E_RoundRobin.iat[0, 1], Group_E_RoundRobin.iat[1, 0] = '7,0' , '0,7'
Group_E_RoundRobin.iat[2, 3], Group_E_RoundRobin.iat[3, 2] = '1,2' , '2,1'

#Group_E_RoundRobin.iat[0, 2], Group_E_RoundRobin.iat[2, 0] = '0,2' , '2,0'
Group_E_RoundRobin.iat[1, 3], Group_E_RoundRobin.iat[3, 1] = Score_inverselist[no] , Score_list[no]

#Group_E_RoundRobin.iat[0, 3], Group_E_RoundRobin.iat[3, 0] = '0,2' , '2,0'
#Group_E_RoundRobin.iat[1, 2], Group_E_RoundRobin.iat[2, 1] = '0,2' , '2,0'
Ranking(Group_E_RoundRobin)
Ranking_num_E, Ranking_probability_E = RankingProbability(Group_E_RoundRobin.copy())
for i in range(1,5):
    Ranking_JapanPropability.at[Score_list[no], i] = Ranking_probability_E.at['Japan', i]
Ranking_JapanPropability.at[Score_list[no], 'Breakthrough'] = Ranking_probability_E.at['Japan', 'Breakthrough']
print(Score_list[no])
Ranking_probability_E

start : 2022.11.25 00:59:21
4,7


,1,2,3,4,Breakthrough
Spain,65.9%,12.1%,17.2%,4.7%,78.0%
CostaRica,5.9%,35.0%,20.6%,38.5%,40.9%
Germany,12.4%,23.0%,22.6%,42.0%,35.4%
Japan,16.2%,29.7%,39.7%,14.4%,45.9%


In [59]:
no = 50
Group_E_RoundRobin.iat[0, 1], Group_E_RoundRobin.iat[1, 0] = '7,0' , '0,7'
Group_E_RoundRobin.iat[2, 3], Group_E_RoundRobin.iat[3, 2] = '1,2' , '2,1'

#Group_E_RoundRobin.iat[0, 2], Group_E_RoundRobin.iat[2, 0] = '0,2' , '2,0'
Group_E_RoundRobin.iat[1, 3], Group_E_RoundRobin.iat[3, 1] = Score_inverselist[no] , Score_list[no]

#Group_E_RoundRobin.iat[0, 3], Group_E_RoundRobin.iat[3, 0] = '0,2' , '2,0'
#Group_E_RoundRobin.iat[1, 2], Group_E_RoundRobin.iat[2, 1] = '0,2' , '2,0'
Ranking(Group_E_RoundRobin)
Ranking_num_E, Ranking_probability_E = RankingProbability(Group_E_RoundRobin.copy())
for i in range(1,5):
    Ranking_JapanPropability.at[Score_list[no], i] = Ranking_probability_E.at['Japan', i]
Ranking_JapanPropability.at[Score_list[no], 'Breakthrough'] = Ranking_probability_E.at['Japan', 'Breakthrough']
print(Score_list[no])
Ranking_probability_E

start : 2022.11.25 01:06:49
3,6


,1,2,3,4,Breakthrough
Spain,66.1%,12.0%,17.2%,4.7%,78.0%
CostaRica,5.9%,35.0%,20.6%,38.5%,40.9%
Germany,12.5%,23.0%,22.6%,41.9%,35.5%
Japan,16.0%,29.8%,39.7%,14.4%,45.8%


In [60]:
no = 51
Group_E_RoundRobin.iat[0, 1], Group_E_RoundRobin.iat[1, 0] = '7,0' , '0,7'
Group_E_RoundRobin.iat[2, 3], Group_E_RoundRobin.iat[3, 2] = '1,2' , '2,1'

#Group_E_RoundRobin.iat[0, 2], Group_E_RoundRobin.iat[2, 0] = '0,2' , '2,0'
Group_E_RoundRobin.iat[1, 3], Group_E_RoundRobin.iat[3, 1] = Score_inverselist[no] , Score_list[no]

#Group_E_RoundRobin.iat[0, 3], Group_E_RoundRobin.iat[3, 0] = '0,2' , '2,0'
#Group_E_RoundRobin.iat[1, 2], Group_E_RoundRobin.iat[2, 1] = '0,2' , '2,0'
Ranking(Group_E_RoundRobin)
Ranking_num_E, Ranking_probability_E = RankingProbability(Group_E_RoundRobin.copy())
for i in range(1,5):
    Ranking_JapanPropability.at[Score_list[no], i] = Ranking_probability_E.at['Japan', i]
Ranking_JapanPropability.at[Score_list[no], 'Breakthrough'] = Ranking_probability_E.at['Japan', 'Breakthrough']
print(Score_list[no])
Ranking_probability_E

start : 2022.11.25 01:14:17
2,5


,1,2,3,4,Breakthrough
Spain,66.2%,11.8%,17.2%,4.7%,78.0%
CostaRica,5.9%,35.0%,20.6%,38.5%,40.9%
Germany,12.5%,23.0%,22.6%,41.8%,35.5%
Japan,15.8%,30.0%,39.7%,14.5%,45.8%


In [61]:
no = 52
Group_E_RoundRobin.iat[0, 1], Group_E_RoundRobin.iat[1, 0] = '7,0' , '0,7'
Group_E_RoundRobin.iat[2, 3], Group_E_RoundRobin.iat[3, 2] = '1,2' , '2,1'

#Group_E_RoundRobin.iat[0, 2], Group_E_RoundRobin.iat[2, 0] = '0,2' , '2,0'
Group_E_RoundRobin.iat[1, 3], Group_E_RoundRobin.iat[3, 1] = Score_inverselist[no] , Score_list[no]

#Group_E_RoundRobin.iat[0, 3], Group_E_RoundRobin.iat[3, 0] = '0,2' , '2,0'
#Group_E_RoundRobin.iat[1, 2], Group_E_RoundRobin.iat[2, 1] = '0,2' , '2,0'
Ranking(Group_E_RoundRobin)
Ranking_num_E, Ranking_probability_E = RankingProbability(Group_E_RoundRobin.copy())
for i in range(1,5):
    Ranking_JapanPropability.at[Score_list[no], i] = Ranking_probability_E.at['Japan', i]
Ranking_JapanPropability.at[Score_list[no], 'Breakthrough'] = Ranking_probability_E.at['Japan', 'Breakthrough']
print(Score_list[no])
Ranking_probability_E

start : 2022.11.25 01:21:44
1,4


,1,2,3,4,Breakthrough
Spain,66.3%,11.8%,17.2%,4.7%,78.1%
CostaRica,5.9%,35.0%,20.6%,38.5%,40.8%
Germany,12.6%,23.0%,22.7%,41.6%,35.6%
Japan,15.6%,30.1%,39.7%,14.6%,45.7%


In [62]:
no = 53
Group_E_RoundRobin.iat[0, 1], Group_E_RoundRobin.iat[1, 0] = '7,0' , '0,7'
Group_E_RoundRobin.iat[2, 3], Group_E_RoundRobin.iat[3, 2] = '1,2' , '2,1'

#Group_E_RoundRobin.iat[0, 2], Group_E_RoundRobin.iat[2, 0] = '0,2' , '2,0'
Group_E_RoundRobin.iat[1, 3], Group_E_RoundRobin.iat[3, 1] = Score_inverselist[no] , Score_list[no]

#Group_E_RoundRobin.iat[0, 3], Group_E_RoundRobin.iat[3, 0] = '0,2' , '2,0'
#Group_E_RoundRobin.iat[1, 2], Group_E_RoundRobin.iat[2, 1] = '0,2' , '2,0'
Ranking(Group_E_RoundRobin)
Ranking_num_E, Ranking_probability_E = RankingProbability(Group_E_RoundRobin.copy())
for i in range(1,5):
    Ranking_JapanPropability.at[Score_list[no], i] = Ranking_probability_E.at['Japan', i]
Ranking_JapanPropability.at[Score_list[no], 'Breakthrough'] = Ranking_probability_E.at['Japan', 'Breakthrough']
print(Score_list[no])
Ranking_probability_E

start : 2022.11.25 01:29:13
0,3


,1,2,3,4,Breakthrough
Spain,66.3%,11.8%,17.2%,4.7%,78.1%
CostaRica,5.9%,35.0%,20.5%,38.7%,40.8%
Germany,12.6%,23.1%,23.0%,41.3%,35.7%
Japan,15.5%,30.2%,39.7%,14.7%,45.7%


In [63]:
no = 54
Group_E_RoundRobin.iat[0, 1], Group_E_RoundRobin.iat[1, 0] = '7,0' , '0,7'
Group_E_RoundRobin.iat[2, 3], Group_E_RoundRobin.iat[3, 2] = '1,2' , '2,1'

#Group_E_RoundRobin.iat[0, 2], Group_E_RoundRobin.iat[2, 0] = '0,2' , '2,0'
Group_E_RoundRobin.iat[1, 3], Group_E_RoundRobin.iat[3, 1] = Score_inverselist[no] , Score_list[no]

#Group_E_RoundRobin.iat[0, 3], Group_E_RoundRobin.iat[3, 0] = '0,2' , '2,0'
#Group_E_RoundRobin.iat[1, 2], Group_E_RoundRobin.iat[2, 1] = '0,2' , '2,0'
Ranking(Group_E_RoundRobin)
Ranking_num_E, Ranking_probability_E = RankingProbability(Group_E_RoundRobin.copy())
for i in range(1,5):
    Ranking_JapanPropability.at[Score_list[no], i] = Ranking_probability_E.at['Japan', i]
Ranking_JapanPropability.at[Score_list[no], 'Breakthrough'] = Ranking_probability_E.at['Japan', 'Breakthrough']
print(Score_list[no])
Ranking_probability_E

start : 2022.11.25 01:36:39
3,7


,1,2,3,4,Breakthrough
Spain,66.5%,11.6%,17.2%,4.8%,78.0%
CostaRica,9.3%,35.3%,23.5%,31.8%,44.7%
Germany,12.8%,22.7%,22.4%,42.0%,35.5%
Japan,11.8%,30.4%,37.0%,20.8%,42.2%


In [64]:
no = 55
Group_E_RoundRobin.iat[0, 1], Group_E_RoundRobin.iat[1, 0] = '7,0' , '0,7'
Group_E_RoundRobin.iat[2, 3], Group_E_RoundRobin.iat[3, 2] = '1,2' , '2,1'

#Group_E_RoundRobin.iat[0, 2], Group_E_RoundRobin.iat[2, 0] = '0,2' , '2,0'
Group_E_RoundRobin.iat[1, 3], Group_E_RoundRobin.iat[3, 1] = Score_inverselist[no] , Score_list[no]

#Group_E_RoundRobin.iat[0, 3], Group_E_RoundRobin.iat[3, 0] = '0,2' , '2,0'
#Group_E_RoundRobin.iat[1, 2], Group_E_RoundRobin.iat[2, 1] = '0,2' , '2,0'
Ranking(Group_E_RoundRobin)
Ranking_num_E, Ranking_probability_E = RankingProbability(Group_E_RoundRobin.copy())
for i in range(1,5):
    Ranking_JapanPropability.at[Score_list[no], i] = Ranking_probability_E.at['Japan', i]
Ranking_JapanPropability.at[Score_list[no], 'Breakthrough'] = Ranking_probability_E.at['Japan', 'Breakthrough']
print(Score_list[no])
Ranking_probability_E

start : 2022.11.25 01:44:13
2.6


,1,2,3,4,Breakthrough
Spain,63.7%,14.1%,17.4%,4.8%,77.8%
CostaRica,4.1%,35.3%,19.9%,40.7%,39.4%
Germany,10.5%,22.7%,22.0%,44.7%,33.2%
Japan,21.9%,27.8%,40.7%,9.7%,49.7%


In [65]:
no = 56
Group_E_RoundRobin.iat[0, 1], Group_E_RoundRobin.iat[1, 0] = '7,0' , '0,7'
Group_E_RoundRobin.iat[2, 3], Group_E_RoundRobin.iat[3, 2] = '1,2' , '2,1'

#Group_E_RoundRobin.iat[0, 2], Group_E_RoundRobin.iat[2, 0] = '0,2' , '2,0'
Group_E_RoundRobin.iat[1, 3], Group_E_RoundRobin.iat[3, 1] = Score_inverselist[no] , Score_list[no]

#Group_E_RoundRobin.iat[0, 3], Group_E_RoundRobin.iat[3, 0] = '0,2' , '2,0'
#Group_E_RoundRobin.iat[1, 2], Group_E_RoundRobin.iat[2, 1] = '0,2' , '2,0'
Ranking(Group_E_RoundRobin)
Ranking_num_E, Ranking_probability_E = RankingProbability(Group_E_RoundRobin.copy())
for i in range(1,5):
    Ranking_JapanPropability.at[Score_list[no], i] = Ranking_probability_E.at['Japan', i]
Ranking_JapanPropability.at[Score_list[no], 'Breakthrough'] = Ranking_probability_E.at['Japan', 'Breakthrough']
print(Score_list[no])
Ranking_probability_E

start : 2022.11.25 01:51:42
1,5


,1,2,3,4,Breakthrough
Spain,66.6%,11.4%,17.2%,4.8%,78.0%
CostaRica,9.3%,35.3%,23.5%,31.8%,44.6%
Germany,12.9%,22.7%,22.6%,41.8%,35.6%
Japan,11.6%,30.5%,36.9%,21.0%,42.1%


In [66]:
no = 57
Group_E_RoundRobin.iat[0, 1], Group_E_RoundRobin.iat[1, 0] = '7,0' , '0,7'
Group_E_RoundRobin.iat[2, 3], Group_E_RoundRobin.iat[3, 2] = '1,2' , '2,1'

#Group_E_RoundRobin.iat[0, 2], Group_E_RoundRobin.iat[2, 0] = '0,2' , '2,0'
Group_E_RoundRobin.iat[1, 3], Group_E_RoundRobin.iat[3, 1] = Score_inverselist[no] , Score_list[no]

#Group_E_RoundRobin.iat[0, 3], Group_E_RoundRobin.iat[3, 0] = '0,2' , '2,0'
#Group_E_RoundRobin.iat[1, 2], Group_E_RoundRobin.iat[2, 1] = '0,2' , '2,0'
Ranking(Group_E_RoundRobin)
Ranking_num_E, Ranking_probability_E = RankingProbability(Group_E_RoundRobin.copy())
for i in range(1,5):
    Ranking_JapanPropability.at[Score_list[no], i] = Ranking_probability_E.at['Japan', i]
Ranking_JapanPropability.at[Score_list[no], 'Breakthrough'] = Ranking_probability_E.at['Japan', 'Breakthrough']
print(Score_list[no])
Ranking_probability_E

start : 2022.11.25 01:59:19
0,4


,1,2,3,4,Breakthrough
Spain,66.6%,11.4%,17.2%,4.8%,78.0%
CostaRica,9.3%,35.3%,23.5%,32.0%,44.6%
Germany,12.9%,22.7%,22.7%,41.6%,35.7%
Japan,11.5%,30.6%,36.9%,21.0%,42.1%


In [67]:
no = 58
Group_E_RoundRobin.iat[0, 1], Group_E_RoundRobin.iat[1, 0] = '7,0' , '0,7'
Group_E_RoundRobin.iat[2, 3], Group_E_RoundRobin.iat[3, 2] = '1,2' , '2,1'

#Group_E_RoundRobin.iat[0, 2], Group_E_RoundRobin.iat[2, 0] = '0,2' , '2,0'
Group_E_RoundRobin.iat[1, 3], Group_E_RoundRobin.iat[3, 1] = Score_inverselist[no] , Score_list[no]

#Group_E_RoundRobin.iat[0, 3], Group_E_RoundRobin.iat[3, 0] = '0,2' , '2,0'
#Group_E_RoundRobin.iat[1, 2], Group_E_RoundRobin.iat[2, 1] = '0,2' , '2,0'
Ranking(Group_E_RoundRobin)
Ranking_num_E, Ranking_probability_E = RankingProbability(Group_E_RoundRobin.copy())
for i in range(1,5):
    Ranking_JapanPropability.at[Score_list[no], i] = Ranking_probability_E.at['Japan', i]
Ranking_JapanPropability.at[Score_list[no], 'Breakthrough'] = Ranking_probability_E.at['Japan', 'Breakthrough']
print(Score_list[no])
Ranking_probability_E

start : 2022.11.25 02:06:56
2,7


,1,2,3,4,Breakthrough
Spain,66.7%,11.3%,17.1%,4.9%,78.0%
CostaRica,12.5%,35.3%,26.4%,25.8%,47.8%
Germany,13.1%,21.9%,22.6%,42.4%,35.0%
Japan,8.1%,31.3%,34.1%,26.5%,39.4%


In [68]:
no = 59
Group_E_RoundRobin.iat[0, 1], Group_E_RoundRobin.iat[1, 0] = '7,0' , '0,7'
Group_E_RoundRobin.iat[2, 3], Group_E_RoundRobin.iat[3, 2] = '1,2' , '2,1'

#Group_E_RoundRobin.iat[0, 2], Group_E_RoundRobin.iat[2, 0] = '0,2' , '2,0'
Group_E_RoundRobin.iat[1, 3], Group_E_RoundRobin.iat[3, 1] = Score_inverselist[no] , Score_list[no]

#Group_E_RoundRobin.iat[0, 3], Group_E_RoundRobin.iat[3, 0] = '0,2' , '2,0'
#Group_E_RoundRobin.iat[1, 2], Group_E_RoundRobin.iat[2, 1] = '0,2' , '2,0'
Ranking(Group_E_RoundRobin)
Ranking_num_E, Ranking_probability_E = RankingProbability(Group_E_RoundRobin.copy())
for i in range(1,5):
    Ranking_JapanPropability.at[Score_list[no], i] = Ranking_probability_E.at['Japan', i]
Ranking_JapanPropability.at[Score_list[no], 'Breakthrough'] = Ranking_probability_E.at['Japan', 'Breakthrough']
print(Score_list[no])
Ranking_probability_E

start : 2022.11.25 02:14:23
1,6


,1,2,3,4,Breakthrough
Spain,66.8%,11.2%,17.1%,4.9%,78.0%
CostaRica,12.4%,35.4%,26.3%,25.9%,47.8%
Germany,13.1%,21.9%,22.7%,42.3%,35.0%
Japan,8.0%,31.3%,34.1%,26.5%,39.4%


In [69]:
no = 60
Group_E_RoundRobin.iat[0, 1], Group_E_RoundRobin.iat[1, 0] = '7,0' , '0,7'
Group_E_RoundRobin.iat[2, 3], Group_E_RoundRobin.iat[3, 2] = '1,2' , '2,1'

#Group_E_RoundRobin.iat[0, 2], Group_E_RoundRobin.iat[2, 0] = '0,2' , '2,0'
Group_E_RoundRobin.iat[1, 3], Group_E_RoundRobin.iat[3, 1] = Score_inverselist[no] , Score_list[no]

#Group_E_RoundRobin.iat[0, 3], Group_E_RoundRobin.iat[3, 0] = '0,2' , '2,0'
#Group_E_RoundRobin.iat[1, 2], Group_E_RoundRobin.iat[2, 1] = '0,2' , '2,0'
Ranking(Group_E_RoundRobin)
Ranking_num_E, Ranking_probability_E = RankingProbability(Group_E_RoundRobin.copy())
for i in range(1,5):
    Ranking_JapanPropability.at[Score_list[no], i] = Ranking_probability_E.at['Japan', i]
Ranking_JapanPropability.at[Score_list[no], 'Breakthrough'] = Ranking_probability_E.at['Japan', 'Breakthrough']
print(Score_list[no])
Ranking_probability_E

start : 2022.11.25 02:21:49
0,5


,1,2,3,4,Breakthrough
Spain,66.9%,11.2%,17.1%,4.9%,78.0%
CostaRica,12.4%,35.3%,26.3%,26.0%,47.7%
Germany,13.1%,22.1%,22.8%,42.0%,35.2%
Japan,8.0%,31.4%,34.1%,26.6%,39.4%


In [70]:
no = 61
Group_E_RoundRobin.iat[0, 1], Group_E_RoundRobin.iat[1, 0] = '7,0' , '0,7'
Group_E_RoundRobin.iat[2, 3], Group_E_RoundRobin.iat[3, 2] = '1,2' , '2,1'

#Group_E_RoundRobin.iat[0, 2], Group_E_RoundRobin.iat[2, 0] = '0,2' , '2,0'
Group_E_RoundRobin.iat[1, 3], Group_E_RoundRobin.iat[3, 1] = Score_inverselist[no] , Score_list[no]

#Group_E_RoundRobin.iat[0, 3], Group_E_RoundRobin.iat[3, 0] = '0,2' , '2,0'
#Group_E_RoundRobin.iat[1, 2], Group_E_RoundRobin.iat[2, 1] = '0,2' , '2,0'
Ranking(Group_E_RoundRobin)
Ranking_num_E, Ranking_probability_E = RankingProbability(Group_E_RoundRobin.copy())
for i in range(1,5):
    Ranking_JapanPropability.at[Score_list[no], i] = Ranking_probability_E.at['Japan', i]
Ranking_JapanPropability.at[Score_list[no], 'Breakthrough'] = Ranking_probability_E.at['Japan', 'Breakthrough']
print(Score_list[no])
Ranking_probability_E

start : 2022.11.25 02:29:19
1,7


,1,2,3,4,Breakthrough
Spain,66.5%,11.5%,16.8%,5.1%,78.0%
CostaRica,14.4%,35.3%,28.3%,22.0%,49.7%
Germany,13.2%,21.0%,22.7%,43.0%,34.3%
Japan,6.2%,32.0%,32.4%,29.4%,38.2%


In [71]:
no = 62
Group_E_RoundRobin.iat[0, 1], Group_E_RoundRobin.iat[1, 0] = '7,0' , '0,7'
Group_E_RoundRobin.iat[2, 3], Group_E_RoundRobin.iat[3, 2] = '1,2' , '2,1'

#Group_E_RoundRobin.iat[0, 2], Group_E_RoundRobin.iat[2, 0] = '0,2' , '2,0'
Group_E_RoundRobin.iat[1, 3], Group_E_RoundRobin.iat[3, 1] = Score_inverselist[no] , Score_list[no]

#Group_E_RoundRobin.iat[0, 3], Group_E_RoundRobin.iat[3, 0] = '0,2' , '2,0'
#Group_E_RoundRobin.iat[1, 2], Group_E_RoundRobin.iat[2, 1] = '0,2' , '2,0'
Ranking(Group_E_RoundRobin)
Ranking_num_E, Ranking_probability_E = RankingProbability(Group_E_RoundRobin.copy())
for i in range(1,5):
    Ranking_JapanPropability.at[Score_list[no], i] = Ranking_probability_E.at['Japan', i]
Ranking_JapanPropability.at[Score_list[no], 'Breakthrough'] = Ranking_probability_E.at['Japan', 'Breakthrough']
print(Score_list[no])
Ranking_probability_E

start : 2022.11.25 02:36:46
0,6


,1,2,3,4,Breakthrough
Spain,66.6%,11.4%,16.9%,5.1%,78.0%
CostaRica,14.3%,35.3%,28.2%,22.2%,49.6%
Germany,13.2%,21.2%,22.8%,42.8%,34.4%
Japan,6.2%,32.0%,32.4%,29.4%,38.2%


In [72]:
no = 63
Group_E_RoundRobin.iat[0, 1], Group_E_RoundRobin.iat[1, 0] = '7,0' , '0,7'
Group_E_RoundRobin.iat[2, 3], Group_E_RoundRobin.iat[3, 2] = '1,2' , '2,1'

#Group_E_RoundRobin.iat[0, 2], Group_E_RoundRobin.iat[2, 0] = '0,2' , '2,0'
Group_E_RoundRobin.iat[1, 3], Group_E_RoundRobin.iat[3, 1] = Score_inverselist[no] , Score_list[no]

#Group_E_RoundRobin.iat[0, 3], Group_E_RoundRobin.iat[3, 0] = '0,2' , '2,0'
#Group_E_RoundRobin.iat[1, 2], Group_E_RoundRobin.iat[2, 1] = '0,2' , '2,0'
Ranking(Group_E_RoundRobin)
Ranking_num_E, Ranking_probability_E = RankingProbability(Group_E_RoundRobin.copy())
for i in range(1,5):
    Ranking_JapanPropability.at[Score_list[no], i] = Ranking_probability_E.at['Japan', i]
Ranking_JapanPropability.at[Score_list[no], 'Breakthrough'] = Ranking_probability_E.at['Japan', 'Breakthrough']
print(Score_list[no])
Ranking_probability_E

start : 2022.11.25 02:44:04
0,7


,1,2,3,4,Breakthrough
Spain,66.0%,12.1%,16.5%,5.5%,78.0%
CostaRica,15.5%,36.7%,28.2%,19.6%,52.2%
Germany,13.3%,19.1%,23.5%,44.1%,32.4%
Japan,5.5%,32.3%,31.7%,30.5%,37.8%


In [75]:
Ranking_JapanPropability

,1,2,3,4,Breakthrough
"7,0",56.0%,41.2%,2.8%,0.0%,97.2%
"7,1",53.8%,42.5%,3.7%,0.0%,96.3%
"6,0",53.5%,42.6%,3.9%,0.0%,96.1%
"7,2",52.4%,42.7%,4.9%,0.0%,95.1%
"6,1",52.2%,42.7%,5.1%,0.0%,94.9%
"5,0",52.0%,42.8%,5.2%,0.0%,94.8%
"7,3",51.8%,42.3%,5.9%,0.0%,94.1%
"6,2",51.6%,42.3%,6.1%,0.0%,93.9%
"5,1",51.5%,42.2%,6.3%,0.0%,93.7%
"4,0",51.5%,42.1%,6.4%,0.0%,93.6%
